# README
Experiments for the interim report (March 2021 - March 2022).

In [1]:
%cd ../../
%load_ext autoreload
%autoreload 2

e:\Mare5\Documents\Šola\IŠRM\ds-smart-lights


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import pprint
from src.v0.experiments import *
from src.evaluate import default_evaluation_metrics

# tempc Experiments

In [3]:
# Evaluation output settings
metrics = [m for m in default_evaluation_metrics if m[0] in ['MSE', 'MedAE']]
summary_params = dict(
    include_support = True,
    include_standard_error=True,
    include_confidence_intervals=True,
    include_device_src = True,
    include_device_uncertainty=False,
    count_error_thresholds=[1,3,5],
    metrics=metrics,
)
summary_params_ = summary_params.copy()
summary_params_.update(dict(
    include_device_src=False, 
    include_support=False, 
    return_fold_summaries=False,
    include_confidence_intervals=False,
    count_errors_threshold=[],
))
pd.set_option('display.max_columns', 64)

In [4]:
def pretty_loss_summary(df):
    summary = {}
    for loss, _ in metrics:
        summary[loss] = '$' + df[f'{loss}_Loss'].map('{:.3f}'.format) + ' \\pm ' + df[f'{loss}_SE'].map('{:.3f}'.format) + '$'
    df = pd.DataFrame(summary)
    return df

def pretty_thresholds_summary(df):
    summary = {}
    for i in summary_params['count_error_thresholds']:
        summary[f'$< {i}$ °C'] = '$' + (1 - df[f'n_thresh_error_{i}_Loss']).map('{:.3f}'.format) + ' \\pm ' + df[f'n_thresh_error_{i}_SE'].map('{:.3f}'.format) + '$'
    df = pd.DataFrame(summary)
    return df

def pretty_summary_2(df):
    summary = {}
    for loss, _ in metrics:
        summary[loss] = '$' + df[f'{loss}_Loss'].map('{:.3f}'.format) + ' \\pm ' + df[f'{loss}_SE'].map('{:.3f}'.format) + '$'
    for i in summary_params['count_error_thresholds']:
        summary[f'$< {i}$ °C'] = '$' + (1 - df[f'n_thresh_error_{i}_Loss']).map('{:.3f}'.format) + '$'
    df = pd.DataFrame(summary)
    return df

def rows_to_columns(df):
    new = []
    for row in df.iterrows():
        new.append(row[1].add_prefix(f'{row[0]}_'))
    return pd.concat(new, axis=0)

def run_experiment(e):
    print('================')
    print(f'{e.name()}: {e.__doc__}')
    print('===Parameters==')
    pprint.pprint(e.get_params())
    print('================')
    folds, summary = e.evaluate(**summary_params)
    print("===Fold mean====")
    print(folds.drop(folds.filter(regex='_CI_|_SE').columns, axis=1).filter(regex='|'.join(summary.index.to_list()), axis=1).mean())
    print("===All folds====")
    print(summary)
    print('================', flush=True)
    return folds, summary

In [9]:
# Generate latex table for report
print(pretty_loss_summary(df).to_latex(escape=False))
print(pretty_thresholds_summary(df).to_latex(escape=False))
print(pretty_summary_2(df).to_latex(escape=False))

\begin{tabular}{lll}
\toprule
{} &                  MSE &              MedAE \\
\midrule
RFExperimentV5_regularized &   $14.144 \pm 0.097$ &  $2.210 \pm 0.011$ \\
RFExperimentV4             &   $14.914 \pm 0.104$ &  $2.181 \pm 0.012$ \\
RFExperimentV5             &   $16.004 \pm 0.112$ &  $2.235 \pm 0.012$ \\
RF_AllVariablesE           &   $17.029 \pm 0.130$ &  $2.202 \pm 0.013$ \\
RidgeExperimentV5          &   $21.916 \pm 0.157$ &  $2.624 \pm 0.014$ \\
RollingMeanOffsetE         &   $31.586 \pm 0.219$ &  $3.452 \pm 0.017$ \\
RidgeExperimentV4          &   $43.806 \pm 0.394$ &  $2.784 \pm 0.017$ \\
RidgeExperimentV3          &   $46.068 \pm 0.376$ &  $3.106 \pm 0.020$ \\
MeanOffsetExperiment       &   $50.713 \pm 0.464$ &  $3.097 \pm 0.019$ \\
RidgeExperimentV2          &   $55.521 \pm 0.466$ &  $2.944 \pm 0.016$ \\
RidgeExperimentV1          &   $55.537 \pm 0.467$ &  $2.944 \pm 0.016$ \\
MeanTempcExperiment        &   $80.277 \pm 0.315$ &  $7.398 \pm 0.026$ \\
MedianTempcExperiment  

## All experiments summary

In [5]:
params = {
    'device_serials': cfg.master_serials,
    'verbose': False,
}
print(params)
df = { }
for e in ALL_EXPERIMENTS:
    e = e(**params)
    r = e.evaluate(**summary_params_)
    # r = r.mean(numeric_only=True)
    # r = pretty_compact_summary(r)
    r = rows_to_columns(r)
    df[e.name()] = r.to_dict()
df = pd.DataFrame(df).transpose()
df = df.sort_values(by='MSE_Loss')
df.round(3)

{'device_serials': ['134225002', '134225010', '134225019'], 'verbose': False}


,MSE_Loss,MSE_SE,MedAE_Loss,MedAE_SE,n_thresh_error_1_Loss,n_thresh_error_1_SE,n_thresh_error_3_Loss,n_thresh_error_3_SE,n_thresh_error_5_Loss,n_thresh_error_5_SE
RFExperimentV5_regularized,14.144,0.097,2.210,0.011,0.755,0.002,0.379,0.002,0.172,0.002
RFExperimentV4,14.914,0.104,2.181,0.012,0.743,0.002,0.376,0.002,0.183,0.002
RFExperimentV5,16.004,0.112,2.235,0.012,0.742,0.002,0.381,0.002,0.188,0.002
RF_AllVariablesE,17.029,0.130,2.202,0.013,0.736,0.002,0.380,0.002,0.190,0.002
HuberE,21.841,0.155,2.613,0.014,0.789,0.002,0.446,0.002,0.245,0.002
RidgeExperimentV5,21.916,0.157,2.624,0.014,0.788,0.002,0.447,0.002,0.240,0.002
RollingMeanOffsetE,31.586,0.219,3.452,0.017,0.845,0.002,0.561,0.002,0.342,0.002
RidgeExperimentV4,44.629,0.394,2.799,0.017,0.786,0.002,0.477,0.002,0.308,0.002
RidgeExperimentV3,46.068,0.376,3.106,0.020,0.807,0.002,0.512,0.002,0.336,0.002
MeanOffsetExperiment,50.713,0.464,3.097,0.019,0.821,0.002,0.512,0.002,0.318,0.002


In [36]:
for serial in cfg.master_serials:
    params = {
        'device_serials': [serial],
        'verbose': False,
    }
    print(params)
    df = { }
    for e in ALL_EXPERIMENTS:
        e = e(**params)
        r = e.evaluate(**summary_params_)
        # r = r.mean(numeric_only=True)
        # r = pretty_compact_summary(r)
        r = rows_to_columns(r)
        df[e.name()] = r.to_dict()
    df = pd.DataFrame(df).transpose()
    df = df.sort_values(by='MSE_Loss')
    print(df.round(2))

{'device_serials': ['134225002'], 'verbose': False}
                            MSE_Loss  MSE_SE  MedAE_Loss  MedAE_SE  \
RF_AllVariablesE               19.85    0.28        2.41      0.02   
RFExperimentV5_regularized     19.99    0.22        2.73      0.03   
RFExperimentV5                 20.01    0.29        2.38      0.02   
RFExperimentV4                 22.20    0.33        2.40      0.03   
RidgeExperimentV5              28.32    0.41        3.18      0.03   
RidgeExperimentV4              29.33    0.58        2.47      0.03   
RidgeExperimentV3              29.53    0.55        2.53      0.02   
RidgeExperimentV2              33.12    0.59        2.77      0.03   
RidgeExperimentV1              33.13    0.59        2.77      0.03   
WindowMeanOffsetE              36.67    0.67        3.81      0.03   
MeanOffsetExperiment           39.31    0.71        3.78      0.03   
LastKTempcMeanE                65.00    0.93        6.10      0.03   
WindowTempcMeanE               65.93  

## Baseline experiments

In [5]:
folds, summary = run_experiment(MeanTempcExperiment())
folds

MeanTempcExperiment: Predict MEAN temperature of merged master devices.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'constant': None, 'quantile': None, 'strategy': 'mean'},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: /home/t440/faks2/ds-smart-lights/data/v0/experiments/MeanTempcExperiment.dump
===Fold mean====
MSE                     80.275849
MedAE                    7.229500
n_thresh_error_1         0.934844
n_thresh_error_3         0.798560
n_thresh_error_5         0.667278
MSE_d25                 86.405327
MedAE_d25                7.687597
n_thresh_error_1_d25     0.943832
n_thresh_error_3_d25     0.826044
n_thresh_error_5_d25     0.700692
MSE_d29                 75.388331
MedAE_d29                6.974706
n_thresh_error_1_d29     0.931508
n_thresh_error_3_d29     0.777022
n_thresh_error_5_d29     0.643006
MSE_d31                 84.541634
MedAE_d31                7.404500
n_thresh_error_1_d31     0

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,62.023553,1.055964,60.132473,64.144354,5.185116,0.137086,4.985116,5.485116,0.872116,0.678201,0.510197,82,168.702800,12.985116,1.000000,1.000000,1.000000,3162,55.794115,4.585116,0.853890,0.648640,0.472486,2738,66.022744,5.785116,0.889335,0.702703,0.539080
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,94.033659,1.464472,91.207058,97.018450,7.743437,0.090534,7.643437,7.943437,0.900201,0.733032,0.649783,0,NaN,NaN,NaN,NaN,NaN,2533,67.607246,6.943437,0.881563,0.685748,0.590209,3449,113.441634,8.343437,0.913888,0.767759,0.693534
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,122.949320,1.460672,120.031404,125.707657,9.492650,0.119990,9.292650,9.692650,0.998663,0.960548,0.834671,2291,118.698226,9.692650,0.996508,0.954605,0.831951,1140,141.207483,11.192650,1.000000,0.959649,0.856140,2551,118.607866,8.392650,1.000000,0.966288,0.827519
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,44.025589,0.802123,42.414945,45.543575,3.697937,0.065811,3.597937,3.797937,0.887736,0.608667,0.396353,2567,41.663000,3.602063,0.879626,0.599143,0.386833,712,45.574164,3.850000,0.932584,0.626404,0.412921,2698,45.864796,3.797937,0.883617,0.613047,0.401038
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,28.063922,0.450761,27.183748,28.953410,4.009474,0.055560,3.890526,4.090526,0.864092,0.590271,0.385657,2678,27.120858,3.790526,0.851755,0.567588,0.362584,1166,41.917999,5.309474,0.909091,0.696398,0.540309,2138,21.689587,3.609474,0.855005,0.560804,0.330215
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,55.499255,0.731600,54.116398,56.845974,5.795461,0.079689,5.695461,5.904539,0.969910,0.791374,0.564193,2860,53.863941,5.695461,0.969580,0.810490,0.568531,1063,51.494338,5.595461,0.973659,0.742239,0.537159,2059,59.838364,5.995461,0.968431,0.790189,0.572122
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,84.302786,0.838141,82.531648,85.899992,7.971765,0.053118,7.871765,8.071765,0.986120,0.947659,0.871906,2460,86.230214,8.071765,0.987398,0.954472,0.883333,1060,75.363519,7.671765,0.980189,0.916038,0.815094,2460,86.227238,8.071765,0.987398,0.954472,0.884959
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,137.347982,0.862186,135.834086,139.048816,11.618227,0.047330,11.518227,11.618227,1.000000,0.995820,0.985788,2129,140.720369,11.618227,1.000000,0.995773,0.986379,1089,128.626114,11.018227,1.000000,0.992654,0.975207,2763,138.187035,11.718227,1.000000,0.997105,0.989504
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,105.413932,0.980401,103.552189,107.324134,9.856748,0.092565,9.656748,9.956748,0.960876,0.902692,0.842836,1414,91.817800,8.756748,0.951202,0.871287,0.787836,1237,98.508811,9.256748,0.942603,0.856912,0.785772,3330,113.752240,10.306748,0.971772,0.933033,0.887387
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,69.098491,0.931482,67.066071,71.059030,6.924183,0.112384,6.624183,7.124183,0.908726,0.777332,0.631394,1229,48.830736,4.975817,0.858421,0.681041,0.498779,1041,47.789522,4.324183,0.841499,0.645533,0.444765,3712,81.784834,8.024183,0.944235,0.846175,0.727640


In [9]:
folds, summary = run_experiment(MedianTempcExperiment())
folds

MedianTempcExperiment: Predict MEDIAN temperature of merged master devices.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'constant': None, 'quantile': None, 'strategy': 'median'},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\MedianTempcExperiment.dump
===Fold mean====
MSE                     82.045958
MedAE                    7.390000
n_thresh_error_1         0.944055
n_thresh_error_3         0.811761
n_thresh_error_5         0.685012
MSE_d25                 88.760320
MedAE_d25                7.816667
n_thresh_error_1_d25     0.946600
n_thresh_error_3_d25     0.827276
n_thresh_error_5_d25     0.707922
MSE_d29                 76.998303
MedAE_d29                7.150000
n_thresh_error_1_d29     0.939764
n_thresh_error_3_d29     0.789860
n_thresh_error_5_d29     0.657786
MSE_d31                 86.468360
MedAE_d31                7.580000
n_thresh_

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,74.600271,1.185311,72.473500,76.943226,6.10,0.135501,5.90,6.40,0.931628,0.730023,0.574557,82,195.146463,14.00,1.000000,1.000000,1.000000,3162,67.328852,5.4,0.923782,0.706199,0.536053,2738,79.387502,6.80,0.938641,0.749452,0.606282
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,107.885874,1.592702,104.788622,111.129854,8.60,0.090534,8.50,8.80,0.923270,0.781177,0.683551,0,NaN,NaN,NaN,NaN,NaN,2533,79.330604,7.8,0.909199,0.740624,0.628504,3449,128.857315,9.20,0.933604,0.810960,0.723978
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,141.885184,1.576991,138.740270,144.873752,10.40,0.119990,10.20,10.60,1.000000,0.995988,0.915914,2291,137.397503,10.60,1.000000,0.990834,0.906155,1140,161.769167,12.1,1.000000,0.999123,0.933333,2551,137.029651,9.30,1.000000,0.999216,0.916895
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,39.358023,0.741585,37.871524,40.751835,3.55,0.056297,3.45,3.65,0.891919,0.555797,0.369751,2567,37.168328,3.45,0.885859,0.548890,0.357616,712,40.715281,3.8,0.928371,0.570225,0.398876,2698,41.083219,3.55,0.888065,0.558562,0.373610
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,26.870074,0.388509,26.129605,27.594890,4.30,0.076751,4.10,4.40,0.873788,0.616015,0.422434,2678,26.112804,4.00,0.868932,0.596714,0.397685,1166,36.775283,5.3,0.897942,0.699828,0.539451,2138,22.416609,4.00,0.866698,0.594481,0.389616
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,62.837551,0.793197,61.321777,64.276633,6.30,0.065286,6.20,6.40,0.968238,0.872284,0.629723,2860,60.921287,6.20,0.967133,0.881469,0.638462,1063,58.733782,6.1,0.968956,0.848542,0.583255,2059,67.617941,6.60,0.969403,0.871782,0.641574
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,86.481406,0.849509,84.687032,88.099572,8.10,0.053118,8.00,8.20,0.986120,0.952007,0.880769,2460,88.442789,8.20,0.987398,0.958130,0.893089,1060,77.383208,7.8,0.980189,0.923585,0.820755,2460,88.440386,8.20,0.987398,0.958130,0.894309
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,128.024882,0.830277,126.565329,129.668598,11.20,0.047330,11.10,11.20,1.000000,0.992811,0.983280,2129,131.244904,11.20,1.000000,0.992954,0.983560,1089,119.678264,10.6,1.000000,0.987144,0.971534,2763,128.833438,11.30,1.000000,0.994933,0.987695
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,93.594436,0.911801,91.854110,95.396072,9.20,0.092565,9.00,9.30,0.967230,0.887811,0.804715,1414,81.074788,8.10,0.958982,0.850778,0.733380,1237,87.422134,8.6,0.956346,0.838319,0.745352,3330,101.203420,9.65,0.974775,0.921922,0.857057
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,58.921877,0.837070,57.129902,60.718408,6.15,0.095446,5.95,6.30,0.898362,0.733701,0.585423,1229,41.334011,4.60,0.851098,0.625712,0.461351,1041,40.846455,4.0,0.832853,0.585014,0.420749,3712,69.814119,7.20,0.932381,0.811153,0.672683


In [10]:
folds, summary = run_experiment(IdentityTempcExperiment())
folds

IdentityTempcExperiment: Copy input temperature to output.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\IdentityTempcExperiment.dump
===Fold mean====
MSE                     102.466322
MedAE                     6.358500
n_thresh_error_1          0.977580
n_thresh_error_3          0.893585
n_thresh_error_5          0.681451
MSE_d25                 193.806105
MedAE_d25                 9.777222
n_thresh_error_1_d25      0.984008
n_thresh_error_3_d25      0.925649
n_thresh_error_5_d25      0.809747
MSE_d29                  86.330614
MedAE_d29                 6.625500
n_thresh_error_1_d29      0.981032
n_thresh_error_3_d29      0.895174
n_thresh_error_5_d29      0.682079
MSE_d31                  51.757911
MedAE_d31                 5.779500
n_thresh_error_1_d31      0.971292
n_thresh_error_3_d31      0.

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,40.349700,0.610486,39.260413,41.572745,5.240,0.044977,5.16000,5.337625,0.954196,0.795219,0.538950,82,66.692030,7.735,0.963415,0.878049,0.804878,3162,38.032094,5.200,0.943074,0.780835,0.530677,2738,42.237281,5.230,0.966764,0.809350,0.540541
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,57.148624,0.706065,55.780179,58.484243,6.345,0.055242,6.24500,6.450000,0.993146,0.952859,0.708626,0,NaN,NaN,NaN,NaN,NaN,2533,49.583147,6.000,0.996447,0.956573,0.701540,3449,62.704829,6.620,0.990722,0.950130,0.713830
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,79.042711,1.024073,76.985553,80.941876,7.360,0.055100,7.26000,7.480000,0.974590,0.921932,0.775660,2291,88.070766,7.710,0.969446,0.909646,0.783937,1140,67.711997,7.160,0.978070,0.914912,0.744737,2551,75.998312,7.180,0.977656,0.936103,0.782046
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,77.876707,1.131162,75.486955,80.021880,7.140,0.061726,7.04000,7.260000,0.986615,0.936423,0.760248,2567,92.661612,7.730,0.985197,0.940397,0.810674,712,72.279343,7.710,0.997191,0.956461,0.752809,2698,65.286816,6.580,0.985174,0.927354,0.714233
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,71.808418,1.088641,69.708946,73.964002,6.980,0.036639,6.92000,7.065000,0.992979,0.952190,0.806419,2678,87.419800,7.640,0.996266,0.965273,0.866692,1166,74.413309,7.265,0.993997,0.941681,0.762436,2138,50.833402,6.245,0.988307,0.941534,0.754911
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,92.232743,1.941623,88.322289,95.874874,6.280,0.060751,6.16000,6.380000,0.951020,0.814610,0.637914,2860,107.887923,6.820,0.964685,0.842657,0.684615,1063,126.596400,7.750,0.977422,0.845720,0.672625,2059,52.746400,5.420,0.918407,0.759592,0.555124
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,165.842235,3.525776,158.687898,172.199456,6.060,0.043741,5.97000,6.140000,0.971405,0.893980,0.662709,2460,292.571608,7.670,0.991463,0.922764,0.804472,1060,175.353465,6.380,0.972642,0.906604,0.666981,2460,35.014527,5.080,0.950813,0.859756,0.519106
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,280.400367,4.292918,271.674153,289.267678,6.610,0.095516,6.42000,6.800000,0.990804,0.936465,0.705902,2129,704.512584,26.960,1.000000,0.999530,0.997651,1089,88.028770,6.150,0.984389,0.921947,0.732782,2763,29.425976,4.890,0.986247,0.893594,0.470503
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,52.770016,1.301602,50.246683,55.427867,5.420,0.040728,5.34475,5.500000,0.979100,0.860057,0.573315,1414,77.741842,6.880,0.992928,0.944130,0.739038,1237,88.778981,6.320,0.991108,0.883589,0.659660,3330,28.790060,4.840,0.968769,0.815616,0.470871
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,107.191696,2.717126,102.270315,112.832887,6.150,0.047768,6.05000,6.240000,0.981946,0.872116,0.644768,1229,226.696783,8.850,0.992677,0.928397,0.795769,1041,82.528639,6.320,0.975985,0.843420,0.596542,3712,74.541505,5.710,0.980065,0.861530,0.608297


In [11]:
folds, summary = run_experiment(MeanOffsetExperiment())
folds

MeanOffsetExperiment: Predict by offsetting mean error.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\MeanOffsetExperiment.dump
===Fold mean====
MSE                      50.713629
MedAE                     3.152802
n_thresh_error_1          0.821280
n_thresh_error_3          0.511651
n_thresh_error_5          0.318129
MSE_d25                 103.068181
MedAE_d25                 5.567531
n_thresh_error_1_d25      0.856340
n_thresh_error_3_d25      0.593050
n_thresh_error_5_d25      0.441055
MSE_d29                  37.640221
MedAE_d29                 3.280850
n_thresh_error_1_d29      0.837557
n_thresh_error_3_d29      0.536148
n_thresh_error_5_d29      0.321070
MSE_d31                  23.185664
MedAE_d31                 2.777439
n_thresh_error_1_d31      0.779734
n_thresh_error_3_d31      0.433657

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,45.511559,0.653110,44.349852,46.826548,5.718435,0.045501,5.638435,5.808435,0.953360,0.826981,0.602307,82,74.348015,8.243435,0.987805,0.902439,0.817073,3162,42.904998,5.668435,0.934851,0.809930,0.590133,2738,47.658146,5.708435,0.973703,0.844412,0.609934
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,31.974297,0.539693,30.906008,32.987983,4.196016,0.055508,4.103391,4.303641,0.950184,0.678368,0.409896,0,NaN,NaN,NaN,NaN,NaN,2533,26.068567,3.846016,0.947098,0.665614,0.372681,3449,36.311558,4.466016,0.952450,0.687736,0.437228
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,34.761763,0.689587,33.441692,36.119747,3.898126,0.045725,3.819016,3.993376,0.878302,0.620194,0.376296,2291,41.924353,4.438126,0.902663,0.685727,0.444784,1140,27.141276,3.738126,0.849123,0.600877,0.344737,2551,31.734660,3.548126,0.869463,0.569973,0.328891
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,26.493128,0.671925,25.182450,27.835552,2.727215,0.053862,2.627215,2.852465,0.790865,0.467626,0.278233,2567,34.869710,3.227215,0.835995,0.529022,0.345150,712,20.879997,2.967215,0.776685,0.488764,0.255618,2698,20.004568,2.272215,0.751668,0.403632,0.220534
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,18.737105,0.646547,17.467058,19.932833,2.061928,0.029742,2.001928,2.120000,0.733032,0.347877,0.177365,2678,26.085954,2.476928,0.784914,0.417476,0.238611,1166,19.624457,2.378072,0.804460,0.405660,0.239280,2138,9.048204,1.480000,0.629093,0.229186,0.066885
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,42.391069,1.185362,40.012863,44.527607,2.966984,0.052237,2.870000,3.094058,0.797225,0.495486,0.328318,2860,48.323094,2.855000,0.807692,0.479720,0.316783,1063,58.247357,3.673016,0.875823,0.578551,0.380997,2059,25.965217,2.786984,0.742108,0.474502,0.317144
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,87.927505,2.253376,83.235384,92.092431,1.880000,0.034725,1.817742,1.955137,0.707358,0.359197,0.247157,2460,166.144282,3.022742,0.773984,0.502033,0.393496,1060,91.757535,2.362742,0.759434,0.428302,0.300943,2460,8.060389,1.317742,0.618293,0.186585,0.077642
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,157.791839,2.593902,152.575548,163.155358,3.134717,0.135444,2.894486,3.435283,0.822103,0.507607,0.447918,2129,408.688252,20.565283,1.000000,0.996712,0.993894,1089,34.265153,1.794717,0.712580,0.329660,0.235996,2763,13.152715,1.724717,0.728194,0.200869,0.110749
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,16.478630,0.703352,15.124602,17.874557,2.454601,0.035181,2.384980,2.525399,0.803210,0.394583,0.124561,1414,21.210852,2.134601,0.758840,0.307638,0.098303,1237,28.894324,3.074601,0.837510,0.508488,0.237672,3330,9.857135,2.424601,0.809309,0.389189,0.093694
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,45.069396,1.577358,42.269704,48.239296,2.490000,0.038699,2.425553,2.575553,0.777165,0.418589,0.189234,1229,106.019118,3.144447,0.855167,0.516680,0.321400,1041,26.618543,3.305553,0.878002,0.545629,0.252642,3712,30.064043,2.045553,0.723060,0.350485,0.127694


In [12]:
folds, summary = run_experiment(LastKTempcMeanE(k=10))
folds

LastKTempcMeanE: Predict by taking the mean of the last k tempc measurements.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'last_k': 10,
 'model_params': {},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\LastKTempcMeanE.dump
===Fold mean====
MSE                     82.032753
MedAE                    7.551000
n_thresh_error_1         0.986324
n_thresh_error_3         0.931939
n_thresh_error_5         0.780082
MSE_d25                 95.827778
MedAE_d25                8.046167
n_thresh_error_1_d25     0.987539
n_thresh_error_3_d25     0.945068
n_thresh_error_5_d25     0.827901
MSE_d29                 87.696602
MedAE_d29                7.875750
n_thresh_error_1_d29     0.987357
n_thresh_error_3_d29     0.942436
n_thresh_error_5_d29     0.805263
MSE_d31                 74.591137
MedAE_d31                7.320200
n_thresh_error_1_d31     0.987738
n_thresh_error_3_d31

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,37.204449,0.453424,36.389875,38.207425,5.3970,0.039363,5.308212,5.471000,0.970578,0.840856,0.570712,82,45.835878,6.6260,1.000000,0.975610,0.853659,3162,34.913332,5.1915,0.951929,0.797596,0.529096,2738,39.591861,5.5590,0.991234,0.886779,0.610299
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,52.420199,0.432476,51.586941,53.276892,6.7125,0.046254,6.614475,6.804075,0.999331,0.981946,0.805082,0,NaN,NaN,NaN,NaN,NaN,2533,48.699233,6.3490,0.999210,0.978681,0.791946,3449,55.152935,6.9550,0.999420,0.984343,0.814729
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,72.097392,0.768677,70.714046,73.570325,7.7080,0.044469,7.624000,7.803262,0.979438,0.945838,0.858743,2291,67.473452,7.5350,0.965517,0.909646,0.816674,1140,76.100901,8.0240,0.971053,0.945614,0.856140,2551,74.460955,7.7310,0.995688,0.978440,0.897687
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,70.690479,0.790649,69.169563,72.265417,7.5610,0.042681,7.459000,7.625525,0.986615,0.963694,0.858792,2567,69.740042,7.4530,0.984418,0.959096,0.853136,712,74.354617,8.1980,1.000000,0.992978,0.908708,2698,70.627804,7.4345,0.985174,0.960341,0.851001
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,62.708906,0.407031,61.954350,63.573164,7.5865,0.033012,7.517475,7.645000,0.994651,0.986794,0.915914,2678,61.921968,7.5245,0.994399,0.985810,0.910754,1166,69.747412,8.2935,0.997427,0.992281,0.933105,2138,59.856016,7.3555,0.993452,0.985033,0.913003
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,83.118075,1.618582,79.932442,86.196942,6.7360,0.053696,6.634113,6.829675,0.956202,0.837513,0.706954,2860,85.554828,6.8025,0.959790,0.851049,0.726923,1063,113.090084,7.4130,0.975541,0.871119,0.733772,2059,64.259716,6.3750,0.941234,0.801360,0.665372
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,118.367987,1.547191,115.194499,121.345344,7.5255,0.062790,7.401737,7.631762,0.996823,0.941973,0.830602,2460,122.953222,7.4385,0.997967,0.939431,0.831301,1060,125.510502,8.4315,0.995283,0.966038,0.842453,2460,110.705082,7.3480,0.996341,0.934146,0.824797
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,186.963137,1.382289,184.574351,189.806144,13.8500,0.039432,13.767900,13.922000,0.994817,0.970072,0.911386,2129,234.430121,15.1410,0.997182,0.991076,0.978394,1089,172.364517,12.7020,0.995409,0.981635,0.954086,2763,156.141815,12.9230,0.992761,0.949330,0.842924
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,46.381678,0.739719,44.904190,47.793837,5.6850,0.036494,5.603425,5.753050,0.985621,0.905367,0.628992,1414,55.870706,6.0950,0.989392,0.942716,0.712164,1237,58.611098,6.3150,0.988682,0.942603,0.750202,3330,37.809523,5.2310,0.982883,0.875676,0.548649
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,90.375233,1.981040,86.488192,94.448873,6.7485,0.051065,6.636425,6.843337,0.999164,0.945336,0.713641,1229,118.669785,7.8000,0.999186,0.951180,0.768104,1041,103.574323,7.8400,0.999039,0.955812,0.753122,3712,77.305659,6.2900,0.999192,0.940463,0.684537


In [13]:
folds, summary = run_experiment(RecentDeviceTempcMeanE())
folds

RecentDeviceTempcMeanE: Predict by taking the mean of the most recent tempc measurements across all devices.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RecentDeviceTempcMeanE.dump
===Fold mean====
MSE                      85.520904
MedAE                     7.440500
n_thresh_error_1          0.981593
n_thresh_error_3          0.917862
n_thresh_error_5          0.752427
MSE_d25                 105.219527
MedAE_d25                 8.176852
n_thresh_error_1_d25      0.982528
n_thresh_error_3_d25      0.936168
n_thresh_error_5_d25      0.812059
MSE_d29                  91.510421
MedAE_d29                 8.054333
n_thresh_error_1_d29      0.986255
n_thresh_error_3_d29      0.931971
n_thresh_error_5_d29      0.782487
MSE_d31                  73.997946
MedAE_d31                 7.181000
n_thresh_error

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,40.009615,0.557234,39.030463,41.192007,5.376667,0.041591,5.287458,5.453500,0.963557,0.817787,0.559345,82,54.379651,7.043333,1.000000,0.987805,0.914634,3162,37.423468,5.170000,0.944655,0.785579,0.525617,2738,42.565881,5.510000,0.984295,0.849890,0.587655
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,55.752093,0.606273,54.554223,56.896158,6.426667,0.048209,6.320000,6.523333,0.996991,0.958041,0.721665,0,NaN,NaN,NaN,NaN,NaN,2533,50.406109,6.116667,0.998026,0.961311,0.707067,3449,59.678269,6.656667,0.996231,0.955639,0.732386
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,74.937139,0.771608,73.456659,76.399464,7.501667,0.054254,7.395792,7.616667,0.976095,0.928118,0.812604,2291,72.538735,7.230000,0.963335,0.898298,0.783937,1140,77.279222,7.875000,0.985965,0.946491,0.821053,2551,76.044457,7.566667,0.983144,0.946688,0.834575
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,71.505070,0.835660,69.880770,73.018140,7.383333,0.051438,7.280000,7.480000,0.987117,0.955496,0.802577,2567,72.722247,7.313333,0.984418,0.952084,0.799766,712,78.881200,8.506667,0.998596,0.985955,0.869382,2698,68.400438,7.121667,0.986657,0.950704,0.787620
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,67.041412,0.679672,65.740250,68.299983,7.326667,0.040140,7.250792,7.406667,0.994651,0.968572,0.861418,2678,69.350209,7.335000,0.995146,0.967513,0.862211,1166,74.119444,8.236667,0.997427,0.978559,0.900515,2138,60.289335,6.985000,0.992516,0.964453,0.839102
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,87.358547,1.713944,84.127980,90.668543,6.730000,0.058678,6.597458,6.839208,0.935807,0.815279,0.666332,2860,91.907169,6.716667,0.937063,0.826573,0.665734,1063,116.654500,7.550000,0.967074,0.859831,0.714017,2059,65.915779,6.443333,0.917921,0.776591,0.642545
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,122.651853,1.584809,119.471054,125.933225,8.106667,0.132252,7.863333,8.363583,0.981940,0.934615,0.812375,2460,139.389707,8.045000,0.982114,0.932114,0.823577,1060,127.692032,9.335000,0.982075,0.943396,0.821698,2460,103.742215,7.786667,0.981707,0.933333,0.797154
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,190.919746,1.556486,187.913571,194.149648,12.853333,0.034141,12.793333,12.925083,0.999498,0.983448,0.931450,2129,260.483402,15.323333,0.999530,0.989197,0.970409,1089,181.160936,13.073333,1.000000,0.984389,0.961433,2763,141.164523,12.040000,0.999276,0.978646,0.889613
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,50.943668,0.817626,49.316400,52.579444,5.826667,0.040007,5.748250,5.896667,0.986624,0.901187,0.642535,1414,60.219726,6.325000,0.990099,0.942716,0.718529,1237,61.240931,6.590000,0.993533,0.938561,0.746160,3330,43.179686,5.360000,0.982583,0.869670,0.571772
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,94.089894,2.028497,90.250449,98.209325,6.873333,0.050865,6.750000,6.965250,0.993648,0.916082,0.713975,1229,125.984901,8.260000,0.991050,0.929211,0.769731,1041,110.246369,8.090000,0.995197,0.935639,0.757925,3712,78.998877,6.340000,0.994073,0.906250,0.683190


In [14]:
folds, summary = run_experiment(WindowTempcMeanE(time_window='60min'))
folds

WindowTempcMeanE: Predict by taking the mean of a time window of tempc measurements.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {},
 'n_splits': 10,
 'random_seed': 0,
 'time_window': '60min'}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\WindowTempcMeanE.dump
===Fold mean====
MSE                     81.247168
MedAE                    7.484490
n_thresh_error_1         0.987394
n_thresh_error_3         0.927325
n_thresh_error_5         0.775753
MSE_d25                 95.710792
MedAE_d25                8.020764
n_thresh_error_1_d25     0.990321
n_thresh_error_3_d25     0.948512
n_thresh_error_5_d25     0.828395
MSE_d29                 83.601209
MedAE_d29                7.718981
n_thresh_error_1_d29     0.985977
n_thresh_error_3_d29     0.934903
n_thresh_error_5_d29     0.802033
MSE_d31                 74.326578
MedAE_d31                7.283406
n_thresh_error_1_d31     0.988750
n_

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,37.641671,0.470914,36.783136,38.677624,5.398125,0.040149,5.320595,5.482847,0.970746,0.835674,0.569375,82,45.574992,6.625417,1.000000,0.987805,0.853659,3162,35.327625,5.181984,0.951929,0.792220,0.526882,2738,40.076470,5.553364,0.991600,0.881300,0.609934
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,53.387908,0.483046,52.437663,54.322491,6.628819,0.041718,6.545274,6.715729,0.999164,0.974925,0.787529,0,NaN,NaN,NaN,NaN,NaN,2533,49.044763,6.345000,0.999210,0.974734,0.774970,3449,56.577581,6.812500,0.999130,0.975065,0.796753
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,70.694773,0.698115,69.489423,72.086030,7.517825,0.035911,7.440909,7.597715,0.985122,0.942661,0.851387,2291,69.586276,7.425000,0.978175,0.928416,0.829769,1140,67.604915,7.526667,0.978070,0.928947,0.840351,2551,73.071097,7.657143,0.994512,0.961584,0.875735
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,70.855042,0.765546,69.418801,72.321145,7.598750,0.038603,7.518854,7.668205,0.989460,0.969215,0.869332,2567,70.803258,7.546667,0.987924,0.964550,0.861706,712,68.295654,8.048295,1.000000,0.997191,0.943820,2698,71.579731,7.467299,0.988139,0.966271,0.856931
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,62.186333,0.412782,61.439570,63.031516,7.521825,0.034706,7.450528,7.593177,0.994818,0.986292,0.912237,2678,61.656993,7.413125,0.994772,0.983570,0.904780,1166,68.830183,8.139583,0.997427,0.993139,0.938250,2138,59.226016,7.326528,0.993452,0.985968,0.907390
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,81.029844,1.550486,77.973021,83.826707,6.871111,0.061883,6.754981,6.990856,0.958877,0.835005,0.705617,2860,86.292237,6.995833,0.966084,0.859091,0.727972,1063,98.178569,6.960615,0.952963,0.816557,0.689558,2059,64.866882,6.665385,0.951918,0.811073,0.682856
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,117.348398,1.532647,114.196206,120.232664,7.395147,0.068927,7.225207,7.509525,0.999498,0.929599,0.848161,2460,122.538228,7.311705,0.999593,0.925203,0.850407,1060,120.348930,8.012222,0.999057,0.956604,0.852830,2460,110.865656,7.208125,0.999593,0.922358,0.843902
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,185.468083,1.407531,182.793611,188.337701,13.679167,0.050553,13.564962,13.775984,0.992476,0.964053,0.898178,2129,233.721841,15.036364,0.996242,0.984030,0.963363,1089,170.212929,12.843636,0.990817,0.979798,0.948577,2763,154.299285,12.875714,0.990228,0.942454,0.828085
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,45.150784,0.718403,43.755838,46.500192,5.603846,0.032071,5.537885,5.661429,0.985120,0.904364,0.612105,1414,54.387421,6.040769,0.990099,0.946252,0.696605,1237,57.286369,6.294118,0.990299,0.948262,0.751011,3330,36.720656,5.128000,0.981081,0.870270,0.524625
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,88.708845,1.973099,84.983619,92.665104,6.630286,0.054006,6.521062,6.722154,0.998663,0.931461,0.703611,1229,116.835883,7.792000,1.000000,0.957689,0.767290,1041,100.882153,7.837692,1.000000,0.961575,0.754083,3712,75.982406,6.140000,0.997845,0.914332,0.668373


In [15]:
folds, summary = run_experiment(WindowMeanOffsetE())
folds

WindowMeanOffsetE: Predict by offsetting mean error of the mean of a time window of tempc measurements.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {},
 'n_splits': 10,
 'random_seed': 0,
 'time_window': '30min'}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\WindowMeanOffsetE.dump
===Fold mean====
MSE                     31.586049
MedAE                    3.701574
n_thresh_error_1         0.844722
n_thresh_error_3         0.560742
n_thresh_error_5         0.341536
MSE_d25                 38.918245
MedAE_d25                3.994485
n_thresh_error_1_d25     0.841781
n_thresh_error_3_d25     0.568632
n_thresh_error_5_d25     0.386283
MSE_d29                 31.170082
MedAE_d29                3.756324
n_thresh_error_1_d29     0.864950
n_thresh_error_3_d29     0.583548
n_thresh_error_5_d29     0.348019
MSE_d31                 27.965295
MedAE_d31                3.578214
n_thresh_error_1

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,43.649985,0.561083,42.640537,44.858084,5.838593,0.042388,5.747476,5.911843,0.960381,0.843196,0.623036,82,54.086604,7.320843,1.000000,0.987805,0.878049,3162,40.932096,5.633093,0.936116,0.801075,0.579696,2738,46.476195,6.000593,0.987217,0.887509,0.665449
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,29.485828,0.401767,28.731168,30.268650,4.376515,0.050461,4.274182,4.468565,0.961886,0.714644,0.416082,0,NaN,NaN,NaN,NaN,NaN,2533,25.849566,4.111515,0.965653,0.692460,0.380576,3449,32.156355,4.595515,0.959119,0.730937,0.442157
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,28.562041,0.465609,27.760323,29.469023,3.939580,0.052597,3.841246,4.046246,0.913574,0.627884,0.391174,2291,27.677058,3.876890,0.914884,0.627674,0.383239,1140,29.156574,4.271121,0.912281,0.657895,0.422807,2551,29.091138,3.881496,0.912975,0.614661,0.384163
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,20.470684,0.440555,19.597497,21.316079,2.966955,0.037076,2.891341,3.032330,0.817802,0.494228,0.239752,2567,20.783317,2.949504,0.819244,0.492014,0.240358,712,17.648592,3.368562,0.853933,0.567416,0.259831,2698,20.917979,2.881855,0.806894,0.477020,0.233877
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,11.140222,0.212095,10.748614,11.579108,2.249721,0.032512,2.196788,2.318888,0.759445,0.378302,0.132564,2678,11.482884,2.225888,0.761016,0.365945,0.138163,1166,13.274173,2.957935,0.816467,0.493139,0.173242,2138,9.547222,2.022638,0.726380,0.331151,0.103368
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,33.354037,0.974030,31.397878,35.261955,3.031890,0.045475,2.939674,3.127361,0.802909,0.504179,0.300903,2860,34.825797,2.862593,0.795455,0.483217,0.286014,1063,48.816626,3.587407,0.848542,0.560677,0.359360,2059,23.326856,3.028502,0.789704,0.504128,0.291404
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,41.320716,0.842090,39.622798,42.967843,2.761000,0.077603,2.629780,2.938156,0.713378,0.481605,0.350000,2460,48.011928,2.760875,0.701220,0.484553,0.358130,1060,43.536343,2.934167,0.786792,0.494340,0.372642,2460,33.674803,2.666517,0.693902,0.473171,0.332114
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,67.057515,0.834799,65.349913,68.784773,7.310245,0.047943,7.212645,7.405146,0.952851,0.846347,0.723290,2129,99.594402,9.208995,0.974636,0.948802,0.894786,1089,51.568625,6.166745,0.943985,0.856749,0.663912,2763,48.091308,6.199412,0.939558,0.763301,0.614549
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,11.001977,0.342363,10.316650,11.612555,2.239574,0.026386,2.184093,2.289345,0.797860,0.334894,0.075071,1414,11.382231,2.062926,0.782885,0.279349,0.059406,1237,11.953925,1.970426,0.761520,0.286176,0.060631,3330,10.486891,2.427093,0.817718,0.376577,0.087087
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,29.817480,1.118749,27.718564,32.078670,2.301667,0.036114,2.230213,2.379579,0.767135,0.382146,0.163490,1229,42.419987,2.681850,0.826688,0.448332,0.238405,1041,28.964302,2.562267,0.824207,0.425552,0.207493,3712,25.884204,2.078517,0.731412,0.348060,0.126347


## Random forest experiments

Notes:
- Encoding time using circular cartesian coordinates seems to work better than using linear time (see: v3 vs v4 and v5 vs v6).  
- Using all weather data and device information seems to overfit? (v4 has better scores than v5/v6) 
  - But it helps in v1, v2 ... 
  - => NEED TO TUNE HYPERPARAMETERS

In [16]:
df = { }
for e in RF_EXPERIMENTS:
    e = e(verbose=False)
    r = e.evaluate(**summary_params_)
    # r = r.mean(numeric_only=True)
    # r = pretty_compact_summary(r)
    r = rows_to_columns(r)
    df[e.name()] = r.to_dict()
df = pd.DataFrame(df).transpose()
df = df.sort_values(by='MSE_Loss')
df.round(2)

,MSE_Loss,MSE_SE,MedAE_Loss,MedAE_SE,n_thresh_error_1_Loss,n_thresh_error_1_SE,n_thresh_error_3_Loss,n_thresh_error_3_SE,n_thresh_error_5_Loss,n_thresh_error_5_SE
RFExperimentV5_regularized,14.31,0.14,2.23,0.01,0.77,0.0,0.38,0.0,0.17,0.0
RFExperimentV4,14.91,0.10,2.18,0.01,0.74,0.0,0.38,0.0,0.18,0.0
RFExperimentV5,16.00,0.11,2.23,0.01,0.74,0.0,0.38,0.0,0.19,0.0
RF_AllVariablesE,17.03,0.13,2.20,0.01,0.74,0.0,0.38,0.0,0.19,0.0
RFExperimentV3,19.01,0.15,2.40,0.01,0.76,0.0,0.41,0.0,0.21,0.0
RFExperimentV6,20.61,0.15,2.49,0.01,0.77,0.0,0.43,0.0,0.23,0.0
RFExperimentV2,24.28,0.22,2.59,0.01,0.78,0.0,0.44,0.0,0.25,0.0
RFExperimentV1,26.58,0.23,2.69,0.01,0.79,0.0,0.46,0.0,0.26,0.0
RFExperimentV0,41.58,0.34,3.07,0.02,0.82,0.0,0.51,0.0,0.31,0.0


In [17]:
folds, summary = run_experiment(RFExperimentV0())
folds

RFExperimentV0: Random forest using tempc only.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RFExperimentV0.dump
===Fold mean====
MSE             

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,77.447900,0.923547,75.635653,79.306570,7.437000,0.083304,7.287239,7.592473,0.937813,0.817118,0.679706,82,21.942709,3.187000,0.768293,0.548780,0.317073,3162,74.171056,7.237000,0.921569,0.782416,0.648008,2738,82.894506,7.837000,0.961651,0.865230,0.727173
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,11.162688,0.246212,10.691214,11.646672,1.933130,0.032441,1.881003,2.000792,0.717987,0.321966,0.129890,0,NaN,NaN,NaN,NaN,NaN,2533,10.069479,1.910000,0.721279,0.300434,0.108962,3449,11.965559,1.960714,0.715570,0.337779,0.145259
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,17.185643,0.398349,16.428881,17.955420,2.305643,0.040052,2.225305,2.377200,0.753594,0.390171,0.205617,2291,19.934846,2.422083,0.762549,0.408555,0.220428,1140,16.399997,2.386381,0.767544,0.399123,0.200000,2551,15.067734,2.164265,0.739318,0.369659,0.194826
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,13.788312,0.329497,13.143541,14.442743,2.060595,0.035327,1.993692,2.128058,0.738498,0.338631,0.153589,2567,15.816520,2.156636,0.746007,0.366965,0.174133,712,11.562291,2.278936,0.780899,0.376404,0.130618,2698,12.446028,1.888945,0.720163,0.301705,0.140104
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,11.097312,0.350504,10.407835,11.772946,1.818263,0.026300,1.755859,1.861527,0.709295,0.275995,0.096958,2678,13.413912,1.946240,0.731889,0.290889,0.101195,1166,14.020941,2.548303,0.789022,0.420240,0.169811,2138,6.601144,1.419551,0.637512,0.178672,0.051918
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,35.676448,0.794120,34.124279,37.246557,3.421608,0.049128,3.321345,3.519878,0.837178,0.551989,0.338683,2860,39.338932,3.347885,0.831119,0.542657,0.332168,1063,44.235088,3.768289,0.873942,0.576670,0.386642,2059,26.170600,3.392386,0.826615,0.552210,0.322972
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,73.306274,1.654194,69.994784,76.440006,3.438737,0.067671,3.305901,3.569457,0.836288,0.541137,0.371739,2460,129.076495,4.129687,0.856098,0.581707,0.460569,1060,70.374275,3.845623,0.850000,0.573585,0.411321,2460,18.799434,2.895874,0.810569,0.486585,0.265854
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,123.473931,1.880409,120.025920,127.102402,5.935755,0.069719,5.823181,6.073013,0.966226,0.841498,0.603411,2129,295.709108,17.142561,0.999530,0.996242,0.994833,1089,43.567078,4.909634,0.950413,0.767677,0.479339,2763,22.254196,4.291004,0.946797,0.751357,0.350706
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,20.947155,0.471221,20.053757,21.925815,3.206884,0.044282,3.147332,3.313872,0.850192,0.531516,0.248119,1414,26.462347,3.569352,0.884017,0.582744,0.309760,1237,33.003124,3.904588,0.877122,0.613581,0.383185,3330,14.126820,2.874275,0.825826,0.479279,0.171772
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,31.739628,0.890122,30.099861,33.560271,2.870752,0.039306,2.796723,2.953996,0.806921,0.479438,0.243063,1229,64.303697,3.733946,0.842962,0.581774,0.375102,1041,25.359589,2.920744,0.813641,0.489914,0.282421,3712,22.747274,2.662482,0.793103,0.442619,0.188308


In [18]:
folds, summary = run_experiment(RFExperimentV1())
folds

RFExperimentV1: RF using basic weather data (tempc, pressmbar, relhumperc).
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RFExperimentV1.dump
===Fo

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,23.141052,0.398543,22.396667,23.923758,3.339000,0.076284,3.205162,3.487000,0.841023,0.541123,0.331494,82,16.958797,2.7720,0.951220,0.487805,0.280488,3162,21.668170,3.156000,0.828906,0.524035,0.317837,2738,25.027173,3.548500,0.851717,0.562454,0.348795
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,13.758714,0.310271,13.209714,14.353555,2.110500,0.032381,2.043950,2.175700,0.737546,0.348211,0.164995,0,NaN,NaN,NaN,NaN,NaN,2533,10.892240,2.011000,0.723253,0.311488,0.124358,3449,15.863898,2.196000,0.748043,0.375181,0.194839
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,18.683185,0.426824,17.769162,19.538333,2.217000,0.045020,2.140663,2.307000,0.750919,0.397359,0.214978,2291,22.283799,2.4390,0.759494,0.417722,0.250546,1140,19.617305,2.674500,0.807018,0.462281,0.215789,2551,15.032105,1.871000,0.718150,0.350059,0.182673
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,14.324604,0.291131,13.760463,14.873954,2.323000,0.038310,2.250000,2.396630,0.765936,0.388155,0.183537,2567,16.322914,2.5000,0.793923,0.423062,0.203740,712,14.370655,2.710667,0.831461,0.439607,0.199438,2698,12.411168,2.025000,0.722016,0.341364,0.160119
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,10.381919,0.291314,9.776380,10.937574,2.000125,0.033793,1.937475,2.068263,0.740555,0.314276,0.098128,2678,10.662356,2.0050,0.746826,0.308813,0.093727,1166,13.528221,2.683833,0.822470,0.444254,0.168096,2138,8.314754,1.658643,0.688026,0.250234,0.065482
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,27.125660,0.602426,25.966148,28.266805,3.166333,0.045263,3.077750,3.260762,0.812103,0.522401,0.285356,2860,29.300659,3.0575,0.829720,0.506643,0.280070,1063,36.323362,3.713000,0.838194,0.580433,0.338664,2059,19.356038,3.129000,0.774162,0.514327,0.265177
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,52.920201,1.417631,49.973199,55.459299,3.237500,0.055044,3.118700,3.340050,0.816221,0.525418,0.345485,2460,84.430052,3.9810,0.852439,0.592276,0.430488,1060,58.913552,3.092500,0.832075,0.514151,0.337736,2460,18.827848,2.715500,0.773171,0.463415,0.263821
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,54.790067,1.265199,52.362982,57.318780,3.808000,0.085011,3.638325,3.966000,0.823274,0.564454,0.414145,2129,107.678691,5.9580,0.844528,0.613434,0.527478,1089,36.699277,4.250000,0.839302,0.593205,0.430670,2763,21.167551,3.140333,0.800579,0.515382,0.320304
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,19.436800,0.447895,18.545508,20.337378,2.627000,0.039758,2.532475,2.686750,0.774118,0.445410,0.238923,1414,22.030510,2.6210,0.776521,0.441301,0.241867,1237,27.296983,3.090000,0.818917,0.512530,0.332255,3330,15.415614,2.467000,0.756456,0.422222,0.203003
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,31.227293,0.749084,29.703921,32.545002,3.310875,0.051033,3.210946,3.415025,0.832832,0.536944,0.311434,1229,35.375483,2.9670,0.825875,0.499593,0.270952,1041,29.250781,3.323000,0.848223,0.543708,0.323727,3712,30.408172,3.421000,0.830819,0.547414,0.321390


In [19]:
folds, summary = run_experiment(RFExperimentV2())
folds

RFExperimentV2: RF using basic weather data (tempc, pressmbar, relhumperc) and device information.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RF

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,26.377550,0.432160,25.621516,27.227747,3.578000,0.077458,3.439113,3.747562,0.822635,0.561351,0.376797,82,18.303387,2.79900,0.939024,0.487805,0.317073,3162,24.232329,3.3005,0.809298,0.536686,0.353257,2738,29.096787,3.970000,0.834551,0.592038,0.405771
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,15.024609,0.333073,14.459925,15.697217,2.276000,0.032455,2.214900,2.341750,0.747910,0.383149,0.180709,0,NaN,NaN,NaN,NaN,NaN,2533,11.457156,2.1670,0.731544,0.345440,0.133044,3449,17.644602,2.364000,0.759930,0.410844,0.215715
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,18.977865,0.435496,18.074801,19.833022,2.199417,0.043792,2.107000,2.271667,0.738382,0.397526,0.215647,2291,22.905232,2.39200,0.749454,0.424269,0.250546,1140,20.006808,2.5965,0.782456,0.452632,0.218421,2551,14.990961,1.881000,0.708742,0.348883,0.183065
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,14.156879,0.295117,13.602872,14.746817,2.287000,0.031851,2.222000,2.352050,0.764430,0.379455,0.182031,2567,16.425187,2.46900,0.797039,0.417998,0.209583,712,13.245977,2.6375,0.818820,0.418539,0.186798,2698,12.239094,2.017792,0.719051,0.332468,0.154559
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,10.125051,0.288635,9.553056,10.654437,1.920000,0.031640,1.864325,1.980000,0.735206,0.300234,0.097125,2678,10.394111,1.86435,0.730769,0.292756,0.097834,1166,12.594145,2.5360,0.831904,0.421098,0.142367,2138,8.441466,1.668750,0.688026,0.243686,0.071562
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,27.357004,0.583025,26.230866,28.489335,3.224500,0.061904,3.117237,3.348575,0.808760,0.527248,0.304915,2860,29.725413,3.17150,0.826224,0.525524,0.313287,1063,36.054655,3.7090,0.817498,0.583255,0.346190,2059,19.576890,3.001333,0.779990,0.500729,0.271977
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,50.511851,1.303241,47.714054,52.760579,3.251000,0.064653,3.141737,3.382813,0.803679,0.527759,0.340301,2460,79.862440,4.13000,0.867073,0.612195,0.443089,1060,56.665500,3.0300,0.805660,0.505660,0.332075,2460,18.509690,2.623500,0.739431,0.452846,0.241057
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,27.217097,0.658271,26.022341,28.545220,2.603000,0.048695,2.508950,2.695000,0.784484,0.452934,0.303461,2129,33.563201,2.30800,0.787694,0.410052,0.295444,1089,40.502772,4.2930,0.836547,0.605142,0.444444,2763,17.090802,2.429333,0.761491,0.425986,0.254072
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,18.327485,0.385749,17.600780,19.056320,2.465000,0.046370,2.391169,2.564925,0.757398,0.423842,0.229393,1414,24.576344,3.15500,0.809760,0.519095,0.311881,1237,25.958388,3.2460,0.808407,0.527082,0.338723,3330,12.839403,1.994000,0.716216,0.345045,0.153754
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,34.758254,1.017825,32.931768,36.825689,2.901000,0.048081,2.815475,2.999921,0.803076,0.487128,0.278001,1229,20.257400,2.59150,0.800651,0.427990,0.203417,1041,27.573108,3.3450,0.861671,0.552354,0.330451,3712,41.574333,2.891500,0.787446,0.488416,0.287985


In [20]:
folds, summary = run_experiment(RFExperimentV3())
folds

RFExperimentV3: RF using tempc and measurement time information.
    
    Time is encoded with two features:
     - Hour: 24-hour float (e.g. 13.4)
     - Month & day: 12-month float (e.g. 0.5 == middle of January)
    
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'ra

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,50.738464,0.663948,49.491159,52.153858,5.866,0.100171,5.647275,6.056675,0.913741,0.731361,0.565196,82,16.120718,2.8360,0.780488,0.463415,0.207317,3162,49.093297,5.4050,0.894687,0.696395,0.534788,2738,53.675160,6.4060,0.939737,0.779766,0.611030
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,17.631110,0.401267,16.869891,18.375234,1.949,0.036029,1.884425,2.021762,0.704447,0.365430,0.215313,0,NaN,NaN,NaN,NaN,NaN,2533,9.619227,1.6700,0.672325,0.291354,0.135018,3449,23.515163,2.2660,0.728037,0.419832,0.274282
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,22.422099,0.493973,21.519193,23.474546,2.621,0.045912,2.532000,2.705050,0.775827,0.448679,0.254764,2291,26.238963,3.0770,0.797032,0.508075,0.313837,1140,22.001017,2.8925,0.808772,0.474561,0.262281,2551,19.182428,2.2460,0.742062,0.383771,0.198354
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,8.072901,0.136911,7.795875,8.327230,1.929,0.033212,1.864000,1.985000,0.727957,0.306676,0.079304,2567,8.712536,2.0790,0.750682,0.331126,0.086872,712,6.850526,1.6135,0.672753,0.269663,0.061798,2698,7.786907,1.8645,0.720904,0.293180,0.076723
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,6.224848,0.142689,5.954538,6.497428,1.465,0.024585,1.422737,1.513000,0.640421,0.194417,0.050986,2678,7.413081,1.5795,0.665795,0.227409,0.066841,1166,7.438733,1.6115,0.664666,0.240995,0.078045,2138,4.074484,1.3080,0.595416,0.127689,0.016370
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,19.223528,0.398481,18.410026,20.027280,2.887,0.029258,2.823000,2.935762,0.800568,0.474423,0.214477,2860,19.165784,2.8735,0.807692,0.469231,0.214685,1063,23.133140,2.9040,0.777987,0.479774,0.240828,2059,17.285320,2.8970,0.802331,0.478873,0.200583
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,16.497666,0.299419,15.947922,17.053958,2.756,0.032939,2.696000,2.815763,0.796321,0.463545,0.214381,2460,16.148788,2.6020,0.782114,0.434146,0.201220,1060,17.420206,2.7730,0.785849,0.476415,0.228302,2460,16.449028,2.9000,0.815041,0.487398,0.221545
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,6.107052,0.116096,5.881021,6.317056,1.417,0.030178,1.354950,1.477100,0.613275,0.214011,0.067380,2129,5.623616,1.4940,0.631282,0.200564,0.053077,1089,6.773932,1.5210,0.643710,0.230487,0.078053,2763,6.216717,1.3110,0.587405,0.217879,0.074195
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,19.465583,0.333290,18.858599,20.070901,3.266,0.036868,3.188000,3.328000,0.839993,0.547734,0.267514,1414,24.697186,3.8985,0.871287,0.637199,0.350071,1237,27.308747,3.8460,0.869038,0.632175,0.353274,3330,14.330605,2.8605,0.815916,0.478378,0.200601
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,23.744569,0.718894,22.520679,25.202959,2.287,0.029541,2.232712,2.350525,0.777834,0.373119,0.162487,1229,48.630969,2.8580,0.823434,0.478438,0.301058,1041,12.053380,2.0790,0.752161,0.341018,0.113353,3712,18.783670,2.2180,0.769935,0.347252,0.130388


In [21]:
folds, summary = run_experiment(RFExperimentV4())
folds

RFExperimentV4: RF using tempc and measurement time information.
    
    Time is encoded with FOUR features:
     - Hour: 24-hour float (e.g. 13.4) -> (x, y) cartesian circle coordinates,
     - Month & day: 12-month float (e.g. 1.5 == middle of January) -> (x, y) cartesian circle coordinates
    
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
             

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,29.931876,0.474485,29.007479,30.840748,3.9395,0.071272,3.768475,4.055262,0.837011,0.582748,0.413574,82,9.262936,2.1400,0.853659,0.280488,0.109756,3162,28.689268,3.5870,0.814358,0.552182,0.391208,2738,31.985923,4.4725,0.862673,0.627100,0.448503
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,16.240029,0.394401,15.432535,16.971912,1.8955,0.037958,1.820738,1.962550,0.717820,0.334504,0.195754,0,NaN,NaN,NaN,NaN,NaN,2533,9.005359,1.6240,0.687722,0.268062,0.122385,3449,21.553284,2.1580,0.739925,0.383300,0.249638
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,13.609543,0.329089,12.987334,14.273401,1.9105,0.032925,1.853000,1.980525,0.701103,0.321130,0.154463,2291,16.448658,2.1550,0.725884,0.367089,0.190310,1140,11.364177,1.9690,0.717544,0.312281,0.117544,2551,12.063209,1.7230,0.671501,0.283810,0.138769
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,8.654749,0.177784,8.292116,8.958739,1.7450,0.026508,1.699000,1.796000,0.723273,0.265852,0.101054,2567,9.298303,1.8680,0.744059,0.287495,0.109466,712,6.336431,1.4780,0.657303,0.210674,0.077247,2698,8.654245,1.7200,0.720904,0.259822,0.099333
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,5.138741,0.116042,4.906501,5.354068,1.3310,0.021335,1.294213,1.373762,0.607155,0.163156,0.038282,2678,5.945776,1.4570,0.636669,0.194922,0.048170,1166,6.848192,1.4705,0.635506,0.227273,0.067753,2138,3.195588,1.1465,0.554724,0.088400,0.009822
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,22.661942,0.437728,21.810647,23.510724,3.3170,0.040091,3.240950,3.384025,0.859746,0.556837,0.285189,2860,22.312336,3.0960,0.855245,0.516084,0.267832,1063,28.544701,3.7140,0.864534,0.644403,0.321731,2059,20.110463,3.3740,0.863526,0.568237,0.290432
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,14.284178,0.291634,13.723265,14.845055,2.3930,0.034146,2.325212,2.466313,0.751338,0.399164,0.168227,2460,15.928679,2.4825,0.767886,0.419106,0.174797,1060,15.868415,2.7245,0.762264,0.467925,0.213208,2460,11.957038,2.1995,0.730081,0.349593,0.142276
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,6.866865,0.180448,6.526974,7.224959,1.4560,0.030598,1.390000,1.515525,0.619462,0.218358,0.061194,2129,5.730225,1.5090,0.630813,0.209488,0.056364,1089,7.201599,1.6570,0.653811,0.228650,0.059688,2763,7.610759,1.3240,0.597177,0.221136,0.065509
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,22.148105,0.333469,21.510366,22.865580,3.7020,0.040722,3.631000,3.793000,0.872095,0.608427,0.307474,1414,25.521287,4.0905,0.905233,0.686704,0.369165,1237,25.105705,3.9230,0.893290,0.658852,0.324980,3330,19.617105,3.3740,0.850150,0.556456,0.274775
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,9.602537,0.207392,9.225927,10.041664,1.9560,0.028819,1.900712,2.008288,0.737713,0.307255,0.103143,1229,10.731168,2.0600,0.775427,0.358828,0.138324,1041,11.048146,2.2070,0.772334,0.370797,0.132565,3712,8.823451,1.8360,0.715517,0.272360,0.083244


In [22]:
folds, summary = run_experiment(RFExperimentV5())
folds

RFExperimentV5: RF using basic weather data, circular measurement time, and device information.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RFExp

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,28.925644,0.482744,28.015908,29.884996,3.5880,0.060356,3.496475,3.714500,0.820127,0.570712,0.378469,82,9.964658,2.3585,0.817073,0.329268,0.109756,3162,27.728089,3.3305,0.795383,0.537634,0.357052,2738,30.876509,4.0010,0.848795,0.616143,0.411249
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,19.760278,0.439418,18.908054,20.603040,2.1800,0.045299,2.098738,2.273575,0.712638,0.396189,0.238883,0,NaN,NaN,NaN,NaN,NaN,2533,12.911532,1.9760,0.691670,0.335965,0.178445,3449,24.790105,2.4260,0.728037,0.440418,0.283271
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,13.949802,0.383970,13.174515,14.691677,1.7060,0.029186,1.651713,1.773525,0.684219,0.286861,0.150953,2291,16.764337,1.8420,0.709297,0.345264,0.203405,1140,12.712647,2.0065,0.735965,0.292105,0.131579,2551,11.974991,1.4840,0.638573,0.232066,0.112505
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,12.253733,0.278383,11.695010,12.773708,2.0910,0.029762,2.028950,2.147050,0.746528,0.318387,0.138531,2567,12.622326,2.1410,0.755746,0.335411,0.146474,712,10.467778,1.9540,0.702247,0.280899,0.099719,2698,12.374348,2.0755,0.749444,0.312083,0.141216
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,5.025907,0.121494,4.795643,5.256818,1.2975,0.020739,1.259738,1.345000,0.611334,0.153628,0.041625,2678,5.806692,1.4280,0.638910,0.181852,0.051904,1166,7.432276,1.5025,0.674957,0.244425,0.082333,2138,2.735557,1.0885,0.542095,0.068756,0.006548
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,22.122685,0.439304,21.326898,23.021158,3.1885,0.039454,3.125738,3.281763,0.841859,0.532932,0.246406,2860,21.828628,2.9310,0.825874,0.489860,0.239161,1063,28.427743,3.5280,0.875823,0.592662,0.280339,2059,19.276026,3.3220,0.846527,0.561923,0.238951
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,15.994611,0.369938,15.265953,16.715971,2.4540,0.026955,2.413000,2.512763,0.778094,0.405351,0.170234,2460,20.592177,2.6540,0.790650,0.435772,0.197154,1060,14.549481,2.4960,0.784906,0.431132,0.188679,2460,12.019744,2.3510,0.762602,0.363821,0.135366
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,7.365066,0.192258,7.004662,7.725616,1.5970,0.024701,1.552950,1.652100,0.652065,0.199632,0.060692,2129,8.093764,1.6890,0.685768,0.180836,0.069516,1089,7.549348,1.6950,0.669421,0.242424,0.070707,2763,6.730942,1.4910,0.619254,0.197249,0.049946
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,25.607618,0.378732,24.880795,26.365227,4.0090,0.048153,3.922900,4.110000,0.857549,0.643538,0.361144,1414,29.642109,4.5080,0.876945,0.719943,0.423621,1237,29.354598,4.3160,0.869846,0.704123,0.390461,3330,22.502578,3.6570,0.844745,0.588589,0.323724
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,9.028163,0.188712,8.698000,9.407665,1.9185,0.033559,1.860000,1.989000,0.718154,0.298562,0.093614,1229,9.821248,1.9140,0.715216,0.314890,0.128560,1041,9.521544,2.1300,0.740634,0.334294,0.113353,3712,8.627217,1.8625,0.712823,0.283136,0.076509


In [23]:
folds, summary = run_experiment(RFExperimentV6())
folds

RFExperimentV6: RF using basic weather data, linear measurement time, and device information.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RFExper

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,49.756116,0.658042,48.509898,51.150000,5.6810,0.094588,5.539738,5.863838,0.910064,0.728185,0.545470,82,15.701099,2.8985,0.731707,0.500000,0.170732,3162,48.113114,5.2165,0.894687,0.694813,0.516762,2738,52.673458,6.2925,0.933163,0.773557,0.589847
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,26.765230,0.483666,25.867031,27.701797,2.6925,0.054960,2.576213,2.806625,0.761953,0.469074,0.324473,0,NaN,NaN,NaN,NaN,NaN,2533,19.668041,2.2670,0.747335,0.411765,0.263719,3449,31.977518,3.1500,0.772688,0.511163,0.369092
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,18.247812,0.443455,17.398011,19.141728,2.2750,0.042877,2.199237,2.356262,0.753260,0.383484,0.199933,2291,20.663466,2.5880,0.774771,0.430380,0.247927,1140,17.321748,2.3510,0.783333,0.392105,0.208772,2551,16.492206,1.9590,0.720502,0.337515,0.152881
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,8.890476,0.161816,8.563143,9.171748,1.8760,0.034555,1.806950,1.938000,0.712063,0.323239,0.093525,2567,9.111316,1.9970,0.736658,0.343202,0.097390,712,7.781920,1.3465,0.589888,0.265449,0.082865,2698,8.972906,1.8905,0.720904,0.319496,0.092661
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,5.881460,0.134262,5.627356,6.139529,1.4635,0.021836,1.417000,1.499000,0.641759,0.183383,0.049816,2678,6.947091,1.5910,0.673264,0.219940,0.063107,1166,7.303617,1.5765,0.686106,0.240995,0.075472,2138,3.771079,1.2205,0.578110,0.106174,0.019177
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,21.433402,0.436225,20.557445,22.310414,2.8480,0.040989,2.769162,2.934050,0.801070,0.478770,0.246072,2860,22.107416,2.7555,0.798252,0.467133,0.254545,1063,26.621427,3.0800,0.778928,0.508937,0.293509,2059,17.818759,2.8600,0.816416,0.479359,0.209811
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,18.251825,0.328888,17.585320,18.895438,2.9905,0.041478,2.905850,3.074262,0.828094,0.497659,0.224916,2460,20.465241,2.8975,0.842276,0.486179,0.237805,1060,17.057659,3.0270,0.819811,0.500943,0.227358,2460,16.552969,3.0490,0.817480,0.507724,0.210976
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,6.900700,0.168506,6.574597,7.228598,1.5160,0.027197,1.473000,1.575050,0.646380,0.206654,0.071560,2129,8.279680,1.6790,0.668859,0.215124,0.068107,1089,6.419451,1.5840,0.653811,0.233242,0.064279,2763,6.027819,1.3920,0.626131,0.189649,0.077090
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,23.918265,0.384205,23.157474,24.629495,3.6310,0.050598,3.540475,3.738525,0.861227,0.581675,0.337235,1414,29.419525,4.0980,0.892504,0.644272,0.404526,1237,32.139643,4.1960,0.893290,0.655618,0.414713,3330,18.528287,3.2000,0.836036,0.527628,0.279880
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,26.055491,0.770379,24.694283,27.677848,2.6215,0.031100,2.557900,2.684763,0.801906,0.429121,0.183383,1229,52.527771,3.0180,0.820179,0.504475,0.318145,1041,15.958727,2.6620,0.782901,0.430355,0.162344,3712,20.122381,2.5255,0.801185,0.403825,0.144666


In [24]:
folds, summary = run_experiment(RFExperimentV5(model_params=dict(max_depth=20, min_samples_split=5, max_features='sqrt')))
folds

RFExperimentV5: RF using basic weather data, circular measurement time, and device information.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': 20,
                  'max_features': 'sqrt',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 5,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RFExper

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,22.629182,0.440783,21.754679,23.486764,3.128099,0.049031,3.038978,3.235983,0.804580,0.516550,0.279338,82,65.629503,6.736931,0.987805,0.878049,0.841463,3162,20.343308,2.789748,0.776407,0.475965,0.249526,2738,23.981229,3.404317,0.831629,0.552593,0.296932
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,21.961366,0.410913,21.174644,22.721929,2.886616,0.067554,2.747123,2.994512,0.746406,0.488298,0.310097,0,NaN,NaN,NaN,NaN,NaN,2533,14.872647,2.264120,0.716147,0.423214,0.253060,3449,27.167432,3.431195,0.768629,0.536097,0.351986
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,7.705037,0.145186,7.436933,7.983105,1.738803,0.025181,1.690650,1.787387,0.694417,0.266466,0.081077,2291,8.809316,1.752417,0.692711,0.288084,0.117852,1140,7.207316,1.885874,0.743860,0.258772,0.069298,2551,6.935730,1.673855,0.673853,0.250490,0.053312
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,11.206696,0.227343,10.789121,11.657309,2.113078,0.029066,2.066645,2.175708,0.770621,0.339301,0.119960,2567,11.356384,2.226327,0.793533,0.355668,0.121543,712,9.827290,1.629993,0.738764,0.244382,0.088483,2698,11.428300,2.165792,0.757228,0.348777,0.126761
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,7.199145,0.158770,6.879162,7.531123,1.737361,0.021060,1.691337,1.782616,0.709462,0.218656,0.060682,2678,8.198020,1.848468,0.735997,0.238237,0.076176,1166,10.856901,2.241142,0.751286,0.340480,0.123499,2138,3.953154,1.449458,0.653414,0.127689,0.007016
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,14.552370,0.271833,14.050185,15.091625,2.168866,0.053901,2.081455,2.284202,0.718154,0.411067,0.211802,2860,14.063399,2.159297,0.725175,0.411888,0.206294,1063,17.490355,2.178384,0.709313,0.409219,0.250235,2059,13.714768,2.210280,0.712967,0.410879,0.199611
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,24.732612,0.499031,23.771051,25.692211,2.505180,0.037953,2.421088,2.580016,0.825920,0.430100,0.269231,2460,34.345106,2.605600,0.846341,0.450407,0.308130,1060,22.996362,2.886437,0.866038,0.485849,0.305660,2460,15.868258,2.275710,0.788211,0.385772,0.214634
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,7.595413,0.129259,7.343033,7.820984,1.734197,0.021952,1.691984,1.776238,0.694700,0.281893,0.074904,2129,10.046714,2.169343,0.747299,0.363551,0.129169,1089,7.863057,1.930028,0.733701,0.303030,0.067952,2763,5.601102,1.458209,0.638798,0.210641,0.035831
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,14.330219,0.217653,13.906472,14.774539,2.881717,0.031448,2.823100,2.946018,0.831299,0.478014,0.201137,1414,15.073851,2.857614,0.833805,0.477369,0.225601,1237,16.043097,3.032159,0.844786,0.508488,0.245756,3330,13.378168,2.835674,0.825225,0.466967,0.174174
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,9.525176,0.161611,9.210009,9.843661,2.187461,0.035178,2.115110,2.253642,0.758943,0.358241,0.106820,1229,9.428295,2.261920,0.772172,0.353133,0.113914,1041,9.012431,2.058433,0.759846,0.345821,0.098943,3712,9.701048,2.209817,0.754310,0.363416,0.106681


In [25]:
folds, summary = run_experiment(RF_AllVariablesE())
folds

RF_AllVariablesE: RF using all variables with circular measurement time.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'bootstrap': True,
                  'ccp_alpha': 0.0,
                  'criterion': 'squared_error',
                  'max_depth': None,
                  'max_features': 'auto',
                  'max_leaf_nodes': None,
                  'max_samples': None,
                  'min_impurity_decrease': 0.0,
                  'min_samples_leaf': 1,
                  'min_samples_split': 2,
                  'min_weight_fraction_leaf': 0.0,
                  'n_estimators': 100,
                  'n_jobs': None,
                  'oob_score': False,
                  'random_state': 0,
                  'verbose': 0,
                  'warm_start': False},
 'n_splits': 10,
 'random_seed': 0}
Loading data...Index(['radarpedholdtime', 'partpolu100gm3', 'radarpedminspdlastperiod',
       'maxpowerprocentage', 'radarvechnumde

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,44.938718,0.706370,43.635157,46.358561,5.0580,0.084695,4.908300,5.227025,0.889669,0.682380,0.504346,82,17.562520,2.3835,0.926829,0.426829,0.170732,3162,43.274871,4.8990,0.875079,0.658760,0.491461,2738,47.680109,5.3565,0.905405,0.717312,0.529218
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,12.712651,0.324944,12.086112,13.366648,1.6255,0.029974,1.570950,1.682000,0.665998,0.293380,0.151454,0,NaN,NaN,NaN,NaN,NaN,2533,7.495095,1.4550,0.636005,0.232925,0.100671,3449,16.544506,1.7710,0.688026,0.337779,0.188750
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,27.929081,0.612652,26.697008,29.134716,2.9000,0.055536,2.782738,3.004525,0.766132,0.487797,0.291541,2291,37.178883,3.3140,0.780445,0.537320,0.347883,1140,35.016314,3.9780,0.826316,0.603509,0.369298,2551,16.454858,2.1160,0.726382,0.391611,0.206194
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,9.504055,0.205985,9.097923,9.877905,1.9260,0.028805,1.874950,1.981525,0.719090,0.299816,0.113937,2567,10.110381,1.9860,0.733931,0.317881,0.119595,712,8.281507,1.9530,0.726124,0.271067,0.102528,2698,9.249799,1.8505,0.703113,0.290215,0.111564
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,4.500798,0.107633,4.286123,4.709672,1.2015,0.019867,1.164000,1.242762,0.572384,0.139418,0.034102,2678,5.100311,1.3105,0.598208,0.170276,0.041449,1166,6.617147,1.3675,0.632933,0.210978,0.070326,2138,2.595673,1.0175,0.507016,0.061740,0.005145
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,21.481397,0.457029,20.607885,22.393045,3.0065,0.040677,2.924000,3.079000,0.824306,0.501170,0.238047,2860,21.591596,2.7805,0.811888,0.459091,0.231818,1063,27.739817,3.4020,0.860771,0.593603,0.286924,2059,18.097294,3.0660,0.822729,0.511899,0.221467
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,16.277848,0.337637,15.635030,16.951505,2.6705,0.034996,2.598900,2.732262,0.772910,0.436288,0.190635,2460,18.745009,2.6850,0.770732,0.441057,0.192276,1060,17.244479,2.8095,0.780189,0.476415,0.226415,2460,13.394171,2.5815,0.771951,0.414228,0.173577
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,6.949904,0.174202,6.621964,7.284564,1.4870,0.020508,1.448000,1.525000,0.638020,0.220030,0.062699,2129,6.397517,1.5210,0.665571,0.195867,0.071865,1089,7.248112,1.6190,0.633609,0.258953,0.059688,2763,7.258005,1.4150,0.618531,0.223308,0.056822
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,17.758653,0.313938,17.209818,18.420862,2.8330,0.034344,2.777000,2.906525,0.807056,0.468484,0.228056,1414,21.251583,3.2310,0.823904,0.534653,0.296322,1237,21.944319,3.2510,0.810024,0.537591,0.276475,3330,14.720613,2.6040,0.798799,0.414715,0.181081
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,8.227419,0.172053,7.906274,8.571050,1.8220,0.030543,1.764000,1.886313,0.703778,0.275995,0.082915,1229,10.469765,1.9990,0.737998,0.305126,0.131001,1041,10.136323,2.0590,0.742555,0.333333,0.115274,3712,6.949669,1.6850,0.681573,0.250269,0.057920


## Linear models

In [6]:
df = { }
for e in LINEAR_MODEL_EXPERIMENTS:
    e = e(verbose=False)
    r = e.evaluate(**summary_params_)
    r = rows_to_columns(r)
    df[e.name()] = r.to_dict()
df = pd.DataFrame(df).transpose()
df = df.sort_values(by='MSE_Loss')
df.round(2)

,MSE_Loss,MSE_SE,MedAE_Loss,MedAE_SE,n_thresh_error_1_Loss,n_thresh_error_1_SE,n_thresh_error_3_Loss,n_thresh_error_3_SE,n_thresh_error_5_Loss,n_thresh_error_5_SE
RidgeExperimentV5,21.92,0.16,2.62,0.01,0.79,0.0,0.45,0.0,0.24,0.0
RidgeExperimentV4,43.81,0.39,2.78,0.02,0.79,0.0,0.47,0.0,0.30,0.0
RidgeExperimentV3,46.07,0.38,3.11,0.02,0.81,0.0,0.51,0.0,0.34,0.0
RidgeExperimentV2,55.52,0.47,2.94,0.02,0.81,0.0,0.49,0.0,0.30,0.0
RidgeExperimentV1,55.54,0.47,2.94,0.02,0.81,0.0,0.49,0.0,0.30,0.0


In [27]:
folds, summary = run_experiment(RidgeExperimentV1())
folds

RidgeExperimentV1: ridge regression using basic weather data(tempc, pressmbar, relhumperc)
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'alpha': 1.0,
                  'copy_X': True,
                  'fit_intercept': True,
                  'max_iter': None,
                  'normalize': 'deprecated',
                  'positive': False,
                  'random_state': 0,
                  'solver': 'auto',
                  'tol': 0.001},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RidgeExperimentV1.dump
===Fold mean====
MSE                      55.534424
MedAE                     3.856004
n_thresh_error_1          0.813525
n_thresh_error_3          0.492692
n_thresh_error_5          0.296239
MSE_d25                 125.563473
MedAE_d25                 6.742658
n_thresh_error_1_d25      0.842740
n_thresh_error_3_d25      0.570801
n_thres

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,239.800348,2.645916,235.186104,245.405275,13.726213,0.074014,13.578719,13.872672,0.949682,0.892845,0.864259,82,548.436059,23.622990,0.987805,0.987805,0.975610,3162,225.336405,13.613215,0.918406,0.835547,0.803605,2738,247.260852,13.753014,0.984660,0.956172,0.930972
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,8.596136,0.182310,8.242364,8.942733,1.791896,0.027252,1.737528,1.843079,0.703778,0.285858,0.089268,0,NaN,NaN,NaN,NaN,NaN,2533,7.573098,1.832069,0.719305,0.278721,0.067509,3449,9.347471,1.753443,0.692375,0.291099,0.105248
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,16.382918,0.347371,15.694968,17.051392,2.242442,0.043493,2.174668,2.345859,0.755600,0.379639,0.201939,2291,19.746736,2.323385,0.786120,0.398516,0.209515,1140,16.907049,2.426408,0.748246,0.406140,0.228947,2551,13.127718,2.088591,0.731478,0.350843,0.183065
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,13.000153,0.866175,11.545396,14.901841,1.903373,0.030888,1.847615,1.969375,0.716413,0.299983,0.128994,2567,16.405732,1.895114,0.702766,0.314764,0.145306,712,10.066723,2.250913,0.780899,0.358146,0.094101,2698,10.534060,1.798297,0.712380,0.270571,0.122683
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,9.497499,0.314456,8.880777,10.091231,1.652595,0.026057,1.604777,1.699525,0.703778,0.224841,0.077900,2678,11.448310,1.751257,0.713592,0.214712,0.078417,1166,12.415558,2.417523,0.807033,0.408233,0.152659,2138,5.462547,1.338645,0.635173,0.137512,0.036483
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,31.830992,0.755295,30.320541,33.204144,3.345673,0.048974,3.258465,3.437048,0.863758,0.549482,0.301571,2860,34.582587,3.258668,0.852448,0.536014,0.297552,1063,44.153035,3.743126,0.864534,0.590781,0.386642,2059,21.647460,3.292466,0.879068,0.546867,0.263235
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,69.149995,1.651816,65.798465,72.346313,2.743403,0.040223,2.662400,2.817703,0.817726,0.465217,0.283445,2460,127.845404,3.355364,0.839837,0.546748,0.419512,1060,66.067436,3.206379,0.819811,0.514151,0.366981,2460,11.782844,2.150150,0.794715,0.362602,0.111382
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,121.716487,1.822962,118.253281,125.389555,5.321491,0.064797,5.230055,5.457482,0.981943,0.850359,0.546564,2129,294.783323,17.251313,0.999530,0.998121,0.994364,1089,43.217629,4.636622,0.976125,0.793388,0.438017,2763,19.300983,4.112570,0.970684,0.758958,0.244300
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,19.255381,0.406000,18.512143,20.034776,3.326444,0.049883,3.225504,3.410671,0.852533,0.544391,0.236917,1414,24.497248,3.715488,0.900283,0.596181,0.284300,1237,29.173972,3.994285,0.854487,0.604689,0.367017,3330,13.345082,2.999938,0.831532,0.500000,0.168468
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,26.114328,0.701343,24.874278,27.501952,2.506514,0.044758,2.427060,2.593672,0.790037,0.434303,0.231528,1229,52.325857,3.510345,0.802278,0.544345,0.382425,1041,20.648930,2.718981,0.826129,0.460134,0.241114,3712,18.968722,2.240623,0.775862,0.390625,0.178879


In [28]:
folds, summary = run_experiment(RidgeExperimentV2())
folds

RidgeExperimentV2: ridge regression using basic weather data(tempc, pressmbar, relhumperc) + data normalization
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'memory': None,
                  'ridge': Ridge(random_state=0),
                  'ridge__alpha': 1.0,
                  'ridge__copy_X': True,
                  'ridge__fit_intercept': True,
                  'ridge__max_iter': None,
                  'ridge__normalize': 'deprecated',
                  'ridge__positive': False,
                  'ridge__random_state': 0,
                  'ridge__solver': 'auto',
                  'ridge__tol': 0.001,
                  'standardscaler': StandardScaler(),
                  'standardscaler__copy': True,
                  'standardscaler__with_mean': True,
                  'standardscaler__with_std': True,
                  'steps': [('standardscaler', StandardScaler()),
                            ('ridge', Ridge(random_state=0))],

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,239.635689,2.644296,235.024233,245.237291,13.721421,0.074047,13.573233,13.867138,0.949682,0.892845,0.864259,82,548.039279,23.614456,0.987805,0.987805,0.975610,3162,225.179860,13.609214,0.918406,0.835547,0.803605,2738,247.093773,13.747515,0.984660,0.956172,0.930972
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,8.597454,0.182319,8.243634,8.944045,1.791371,0.027261,1.737612,1.842838,0.703778,0.285858,0.089101,0,NaN,NaN,NaN,NaN,NaN,2533,7.574584,1.832204,0.719305,0.278721,0.067509,3449,9.348667,1.752331,0.692375,0.291099,0.104958
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,16.383940,0.347376,15.695972,17.052395,2.242969,0.043464,2.175036,2.346061,0.755600,0.379639,0.201939,2291,19.747743,2.323874,0.786120,0.398516,0.209515,1140,16.908991,2.427036,0.748246,0.406140,0.228947,2551,13.128342,2.088927,0.731478,0.350843,0.183065
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,13.000467,0.866484,11.545242,14.902881,1.903506,0.030881,1.847353,1.969278,0.716413,0.299983,0.128994,2567,16.406359,1.895212,0.702766,0.314764,0.145306,712,10.066830,2.251039,0.780899,0.358146,0.094101,2698,10.534131,1.798350,0.712380,0.270571,0.122683
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,9.497309,0.314441,8.880612,10.091018,1.652602,0.026051,1.604847,1.699448,0.703778,0.224841,0.077900,2678,11.447883,1.751359,0.713592,0.214712,0.078417,1166,12.415583,2.417439,0.807033,0.408233,0.152659,2138,5.462536,1.338707,0.635173,0.137512,0.036483
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,31.830778,0.755279,30.320349,33.203901,3.345774,0.048979,3.258529,3.437221,0.863758,0.549482,0.301571,2860,34.582232,3.258591,0.852448,0.536014,0.297552,1063,44.152613,3.743110,0.864534,0.590781,0.386642,2059,21.647549,3.292668,0.879068,0.546867,0.263235
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,69.149591,1.651794,65.798109,72.345869,2.743486,0.040237,2.662306,2.817792,0.817726,0.465217,0.283445,2460,127.844130,3.355483,0.839837,0.546748,0.419512,1060,66.066855,3.206471,0.819811,0.514151,0.366981,2460,11.783385,2.150159,0.794715,0.362602,0.111382
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,121.716757,1.822951,118.253588,125.389798,5.321622,0.064779,5.230020,5.457641,0.981943,0.850359,0.546564,2129,294.782310,17.251283,0.999530,0.998121,0.994364,1089,43.218562,4.636791,0.976125,0.793388,0.438017,2763,19.301979,4.112744,0.970684,0.758958,0.244300
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,19.256192,0.406003,18.512964,20.035583,3.326543,0.049888,3.225542,3.410850,0.852533,0.544391,0.236917,1414,24.497985,3.715555,0.900283,0.596181,0.284300,1237,29.174754,3.994429,0.854487,0.604689,0.367017,3330,13.345935,2.999780,0.831532,0.500000,0.168468
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,26.114607,0.701341,24.874557,27.502215,2.506605,0.044767,2.427140,2.593763,0.790037,0.434303,0.231528,1229,52.325938,3.510399,0.802278,0.544345,0.382425,1041,20.649109,2.718856,0.826129,0.460134,0.241114,3712,18.969095,2.240480,0.775862,0.390625,0.178879


In [29]:
folds, summary = run_experiment(RidgeExperimentV3())
folds

RidgeExperimentV3: ridge regression using basic weather data(tempc, pressmbar, relhumperc) + hour and month
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'alpha': 1.0,
                  'copy_X': True,
                  'fit_intercept': True,
                  'max_iter': None,
                  'normalize': 'deprecated',
                  'positive': False,
                  'random_state': 0,
                  'solver': 'auto',
                  'tol': 0.001},
 'n_splits': 10,
 'random_seed': 0}
Loading data...DONE
Loading results from: E:\Mare5\Documents\Šola\IŠRM\ds-smart-lights\data\experiments\RidgeExperimentV3.dump
===Fold mean====
MSE                     46.065506
MedAE                    4.391242
n_thresh_error_1         0.807309
n_thresh_error_3         0.511657
n_thresh_error_5         0.335647
MSE_d25                 94.181250
MedAE_d25                6.328862
n_thresh_error_1_d25     0.834489
n_thresh_error_3_d25     0.569205

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,201.483492,2.231502,197.522511,206.634524,12.493903,0.096111,12.312578,12.706392,0.955533,0.882481,0.854229,82,490.699726,22.431668,1.000000,0.987805,0.975610,3162,190.684198,12.363797,0.926312,0.827641,0.795383,2738,205.293439,12.463393,0.987947,0.942659,0.918554
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,8.364558,0.157954,8.070461,8.686706,1.834955,0.031440,1.775613,1.895568,0.712638,0.302909,0.080909,0,NaN,NaN,NaN,NaN,NaN,2533,7.135613,1.616490,0.671141,0.263324,0.057639,3449,9.267114,2.020869,0.743114,0.331980,0.097999
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,12.350888,0.231235,11.912964,12.826904,2.271069,0.033605,2.210937,2.341206,0.757105,0.378469,0.150451,2291,15.025114,2.324256,0.779136,0.391096,0.177215,1140,12.125399,2.465597,0.772807,0.413158,0.156140,2551,10.049989,2.167020,0.730302,0.351627,0.123873
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,11.454744,1.386188,9.273035,14.503566,2.173710,0.030473,2.121353,2.237838,0.759244,0.358876,0.099883,2567,14.565956,2.253992,0.764316,0.365407,0.101286,712,6.282361,1.584565,0.675562,0.237360,0.057584,2698,9.859584,2.291459,0.776501,0.384729,0.109711
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,5.561917,0.173570,5.231305,5.887408,1.302561,0.018177,1.268962,1.341702,0.602808,0.150786,0.036610,2678,7.293408,1.369081,0.617999,0.172143,0.057506,1166,5.707282,1.661302,0.698113,0.206690,0.032590,2138,3.313821,1.096047,0.531805,0.093545,0.012629
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,20.218081,0.431745,19.382426,21.115304,2.539100,0.036753,2.471548,2.611233,0.783517,0.428285,0.215480,2860,21.311400,2.655108,0.800350,0.444056,0.218881,1063,26.457193,2.842361,0.795861,0.476011,0.271872,2059,15.478368,2.114963,0.753764,0.381739,0.181642
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,24.327858,0.598275,23.171246,25.575552,2.109552,0.040066,2.050177,2.199293,0.733445,0.364047,0.212709,2460,43.362786,2.611650,0.767073,0.448374,0.344715,1060,19.774072,2.335416,0.736792,0.383962,0.235849,2460,7.255131,1.780283,0.698374,0.271138,0.070732
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,103.422781,1.071090,101.220413,105.542100,9.316403,0.073950,9.192465,9.460625,0.979100,0.932620,0.804548,2129,177.995444,13.025187,1.000000,1.000000,0.991076,1089,80.411768,8.560814,0.982553,0.937557,0.781451,2763,55.031103,6.684256,0.961636,0.878755,0.669924
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,59.138112,0.624645,57.921876,60.342493,7.100804,0.059083,6.994179,7.214815,0.967397,0.856546,0.701722,1414,64.289154,7.511395,0.970297,0.865629,0.706506,1237,68.320948,7.823958,0.974131,0.887631,0.767179,3330,53.539691,6.670649,0.963664,0.841141,0.675375
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,14.332628,0.211212,13.916704,14.719441,2.770364,0.033644,2.709392,2.841984,0.822300,0.461551,0.199933,1229,13.088265,2.777422,0.811229,0.448332,0.176566,1041,10.756137,2.348943,0.781940,0.379443,0.115274,3712,15.747620,2.921566,0.837284,0.488955,0.231412


In [30]:
folds, summary = run_experiment(RidgeExperimentV4())
folds

RidgeExperimentV4: Ridge regression using rolling window averaged basic weather data and one-hot hour & month.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'memory': None,
                  'ridge': Ridge(random_state=0),
                  'ridge__alpha': 1.0,
                  'ridge__copy_X': True,
                  'ridge__fit_intercept': True,
                  'ridge__max_iter': None,
                  'ridge__normalize': 'deprecated',
                  'ridge__positive': False,
                  'ridge__random_state': 0,
                  'ridge__solver': 'auto',
                  'ridge__tol': 0.001,
                  'standardscaler': StandardScaler(),
                  'standardscaler__copy': True,
                  'standardscaler__with_mean': True,
                  'standardscaler__with_std': True,
                  'steps': [('standardscaler', StandardScaler()),
                            ('ridge', Ridge(random_state=0))],


,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,251.215528,2.649120,246.349020,257.239927,14.078588,0.115451,13.910618,14.343430,0.956369,0.882648,0.856737,82,531.920253,22.822569,1.000000,1.000000,1.000000,3162,230.068018,13.480717,0.929475,0.827957,0.794434,2738,267.231102,14.631547,0.986121,0.942294,0.924397
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,7.228241,0.136695,6.960488,7.508943,1.703344,0.024169,1.661394,1.754628,0.696423,0.263290,0.068372,0,NaN,NaN,NaN,NaN,NaN,2533,6.065408,1.550214,0.671536,0.225030,0.045401,3449,8.082243,1.824774,0.714700,0.291389,0.085242
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,9.984911,0.189167,9.600895,10.374859,2.036264,0.032362,1.970356,2.096140,0.734203,0.329990,0.108158,2291,11.894191,2.200715,0.763422,0.362724,0.133130,1140,10.223434,1.901263,0.709649,0.314912,0.136842,2551,8.163635,1.952861,0.718934,0.307330,0.072913
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,7.876071,0.128238,7.628204,8.132853,1.933056,0.024969,1.886550,1.983466,0.747867,0.296637,0.059896,2567,8.060731,1.976228,0.758473,0.305415,0.058434,712,4.686187,1.447275,0.630618,0.154494,0.018258,2698,8.542184,2.055110,0.768718,0.325797,0.072276
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,3.422945,0.080358,3.276064,3.605593,1.053168,0.015863,1.022438,1.083531,0.518890,0.101973,0.016550,2678,3.912751,1.124105,0.539955,0.116878,0.024272,1166,2.941020,0.920786,0.473413,0.084906,0.008576,2138,3.072256,1.050719,0.517306,0.092610,0.011225
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,18.098441,0.403511,17.288961,18.870408,2.421780,0.030152,2.369463,2.494222,0.785189,0.404881,0.181043,2860,17.748591,2.434997,0.788112,0.396503,0.175874,1063,23.545986,2.523370,0.807150,0.440263,0.216369,2059,15.771985,2.361032,0.769791,0.398252,0.169985
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,18.632932,0.356657,17.907598,19.320714,2.102910,0.027372,2.051857,2.160130,0.731438,0.375084,0.252676,2460,20.851093,2.113205,0.734959,0.376829,0.267073,1060,18.411881,2.197302,0.723585,0.403774,0.266038,2460,16.510021,2.078092,0.731301,0.360976,0.232520
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,76.212222,0.688844,74.918034,77.572033,8.237501,0.059258,8.124673,8.373135,0.974753,0.912724,0.790169,2129,92.508543,9.359279,0.996712,0.953030,0.880695,1089,78.920774,8.655560,0.985308,0.946740,0.824610,2763,62.587726,7.214790,0.953674,0.868259,0.706840
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,30.326172,0.368471,29.635987,31.029247,4.798875,0.046461,4.721892,4.897453,0.925598,0.732821,0.473499,1414,32.970163,5.106979,0.939180,0.759547,0.510608,1237,35.324570,5.394913,0.958771,0.795473,0.564268,3330,27.346706,4.427748,0.907508,0.698198,0.424024
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,15.029435,0.249089,14.546718,15.489053,2.653997,0.035751,2.595576,2.732287,0.779840,0.442494,0.213139,1229,12.591409,2.354378,0.747762,0.366151,0.159479,1041,10.648681,2.291043,0.756004,0.341979,0.115274,3712,17.065184,2.974331,0.797144,0.495959,0.258351


In [31]:
folds, summary = run_experiment(RidgeExperimentV5())
folds

RidgeExperimentV5: Ridge regression using rolling window averaged basic weather data and circular time.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'memory': None,
                  'ridge': Ridge(random_state=0),
                  'ridge__alpha': 1.0,
                  'ridge__copy_X': True,
                  'ridge__fit_intercept': True,
                  'ridge__max_iter': None,
                  'ridge__normalize': 'deprecated',
                  'ridge__positive': False,
                  'ridge__random_state': 0,
                  'ridge__solver': 'auto',
                  'ridge__tol': 0.001,
                  'standardscaler': StandardScaler(),
                  'standardscaler__copy': True,
                  'standardscaler__with_mean': True,
                  'standardscaler__with_std': True,
                  'steps': [('standardscaler', StandardScaler()),
                            ('ridge', Ridge(random_state=0))],
       

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,61.272395,0.956887,59.603465,63.330849,5.905465,0.070175,5.772301,6.054738,0.893514,0.739218,0.571715,82,197.466736,13.116588,1.000000,1.000000,0.975610,3162,55.640135,5.377104,0.871284,0.692283,0.526882,2738,63.697986,6.199143,0.915997,0.785610,0.611395
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,52.733246,0.691266,51.432276,54.052033,5.904882,0.092976,5.731614,6.075346,0.945670,0.767469,0.577399,0,NaN,NaN,NaN,NaN,NaN,2533,39.729279,5.137579,0.937623,0.727596,0.512436,3449,62.283565,6.519324,0.951580,0.796753,0.625109
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,9.493709,0.211724,9.112109,9.930034,1.985835,0.038223,1.909818,2.058683,0.708793,0.322133,0.080742,2291,9.374437,2.057357,0.713662,0.342209,0.088171,1140,9.579445,2.124635,0.743860,0.328070,0.054386,2551,9.562510,1.870166,0.688750,0.301450,0.085849
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,8.081787,0.168891,7.753100,8.396403,1.876444,0.024382,1.829851,1.928588,0.733144,0.265685,0.072779,2567,8.202028,1.878268,0.733931,0.270354,0.075185,712,6.749473,1.963485,0.769663,0.245787,0.043539,2698,8.318981,1.856578,0.722758,0.266494,0.078206
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,4.316876,0.083814,4.159139,4.496963,1.394284,0.019851,1.355969,1.434037,0.642929,0.146439,0.018723,2678,4.677753,1.430598,0.656460,0.153846,0.025765,1166,3.801330,1.291804,0.615780,0.132933,0.006861,2138,4.146015,1.396258,0.640786,0.144528,0.016370
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,20.024668,0.448867,19.143777,20.902082,2.791205,0.029537,2.728595,2.844123,0.820796,0.458542,0.200435,2860,20.180179,2.707761,0.813636,0.446154,0.200000,1063,26.644838,3.114900,0.844779,0.523048,0.241769,2059,16.390865,2.721011,0.818358,0.442448,0.179699
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,19.287055,0.362114,18.578889,20.004727,2.094566,0.040380,2.033653,2.183361,0.719732,0.388963,0.255017,2460,21.424301,2.132620,0.732114,0.391463,0.271138,1060,19.510451,2.067936,0.689623,0.415094,0.272642,2460,17.053549,2.082131,0.720325,0.375203,0.231301
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,19.309475,0.239697,18.831218,19.750979,3.623486,0.048136,3.526465,3.725261,0.839659,0.579669,0.326701,2129,24.552298,4.321720,0.882574,0.658995,0.425082,1089,18.271037,3.652343,0.820937,0.585859,0.303949,2763,15.678961,3.130049,0.813970,0.516106,0.259862
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,13.229868,0.253238,12.756895,13.723579,2.325663,0.039774,2.250813,2.407474,0.778632,0.395084,0.161511,1414,11.846773,2.399182,0.787836,0.405233,0.137199,1237,10.428941,2.071278,0.750202,0.341956,0.113177,3330,14.857629,2.448319,0.785285,0.410511,0.189790
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,11.397834,0.177697,11.069800,11.729971,2.493200,0.032439,2.433964,2.559981,0.800401,0.401705,0.139084,1229,13.311474,2.811324,0.829943,0.465419,0.165989,1041,12.453398,2.804392,0.853026,0.456292,0.136407,3712,10.468226,2.306243,0.775862,0.365302,0.130927


In [14]:
from sklearn.linear_model import RANSACRegressor
class RANSAC_E(RidgeExperimentV5):
    """RANSAC regression using rolling window averaged basic weather data and one-hot hour & month."""

    def create_model(self):
        r = RANSACRegressor(random_state=self.params['random_seed'], **self.params['model_params'])
        return make_pipeline(StandardScaler(), r)

folds, summary = run_experiment(RANSAC_E())
folds

RANSAC_E: RANSAC regression using rolling window averaged basic weather data and one-hot hour & month.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'memory': None,
                  'ransacregressor': RANSACRegressor(random_state=0),
                  'ransacregressor__base_estimator': None,
                  'ransacregressor__is_data_valid': None,
                  'ransacregressor__is_model_valid': None,
                  'ransacregressor__loss': 'absolute_error',
                  'ransacregressor__max_skips': inf,
                  'ransacregressor__max_trials': 100,
                  'ransacregressor__min_samples': None,
                  'ransacregressor__random_state': 0,
                  'ransacregressor__residual_threshold': None,
                  'ransacregressor__stop_n_inliers': inf,
                  'ransacregressor__stop_probability': 0.99,
                  'ransacregressor__stop_score': inf,
                  'standard

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,102.704420,1.063314,100.915018,105.153667,9.334593,0.095394,9.151722,9.533772,0.963223,0.868773,0.799733,82,234.869662,15.004379,1.000000,1.000000,1.000000,3162,94.544925,8.787653,0.940228,0.816572,0.743201,2738,108.169276,9.761073,0.988678,0.925128,0.859021
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,50.139367,0.635552,48.990322,51.397557,5.938698,0.060995,5.828878,6.067756,0.939652,0.787696,0.595286,0,NaN,NaN,NaN,NaN,NaN,2533,37.500101,5.239485,0.926569,0.748914,0.529412,3449,59.421843,6.598104,0.949261,0.816179,0.643665
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,11.362527,0.213347,10.943296,11.814994,2.221740,0.032869,2.158780,2.285104,0.733868,0.355400,0.144600,2291,12.698242,2.271844,0.738542,0.362287,0.159319,1140,11.373986,2.105592,0.685088,0.341228,0.150000,2551,10.157828,2.226107,0.751470,0.355547,0.128969
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,9.609788,0.232628,9.142052,10.064619,1.973500,0.026767,1.914381,2.020851,0.741509,0.288439,0.090681,2567,9.822704,1.988122,0.738605,0.293339,0.099727,712,7.920978,2.053835,0.806180,0.290730,0.053371,2698,9.852887,1.932951,0.727205,0.283173,0.091920
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,4.980762,0.118691,4.762362,5.223895,1.380546,0.024317,1.323312,1.423971,0.611668,0.149281,0.030926,2678,5.765730,1.500117,0.647872,0.170650,0.038462,1166,3.054282,1.084863,0.530017,0.090909,0.006003,2138,5.048176,1.397891,0.610851,0.154350,0.035080
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,16.566985,0.374043,15.838373,17.314606,2.613181,0.042000,2.536936,2.681509,0.806586,0.431461,0.174356,2860,16.556531,2.531271,0.801049,0.415385,0.171329,1063,22.547739,2.939319,0.833490,0.492004,0.219191,2059,13.493823,2.564923,0.800389,0.422535,0.155415
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,29.022412,0.563134,27.853136,30.100910,2.392600,0.043419,2.294970,2.461794,0.773077,0.424415,0.301839,2460,33.693921,2.391134,0.777642,0.432114,0.323171,1060,26.576894,2.498554,0.765094,0.431132,0.293396,2460,25.404664,2.347982,0.771951,0.413821,0.284146
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,23.638635,0.292019,23.061737,24.175717,4.127928,0.039707,4.059353,4.201584,0.870757,0.647885,0.386892,2129,32.051695,5.150987,0.925317,0.757633,0.520902,1089,21.046486,4.101426,0.842975,0.638200,0.341598,2763,18.177704,3.491127,0.839667,0.567137,0.301484
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,10.984830,0.190140,10.638328,11.368237,2.325477,0.027802,2.283181,2.387014,0.798696,0.362147,0.122555,1414,9.279024,2.243162,0.779349,0.326025,0.087694,1237,8.587364,2.050389,0.754244,0.295069,0.076799,3330,12.599747,2.485344,0.823423,0.402402,0.154354
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,12.204634,0.191190,11.859320,12.561929,2.562033,0.031043,2.499444,2.617702,0.815112,0.416082,0.156971,1229,13.983443,2.827229,0.832384,0.471115,0.192840,1041,13.195602,2.814531,0.842459,0.453410,0.163305,3712,11.337781,2.365472,0.801724,0.387392,0.143319


In [9]:
from sklearn.linear_model import HuberRegressor
class HuberE(RidgeExperimentV5):
    """Huber regression using rolling window averaged basic weather data and one-hot hour & month."""

    def create_model(self):
        r = HuberRegressor(epsilon=2, **self.params['model_params'])
        return make_pipeline(StandardScaler(), r)

folds, summary = run_experiment(HuberE())
folds

HuberE: Huber regression using rolling window averaged basic weather data and one-hot hour & month.
===Parameters==
{'device_serials': ['134225002', '134225010', '134225019'],
 'model_params': {'huberregressor': HuberRegressor(epsilon=3),
                  'huberregressor__alpha': 0.0001,
                  'huberregressor__epsilon': 3,
                  'huberregressor__fit_intercept': True,
                  'huberregressor__max_iter': 100,
                  'huberregressor__tol': 1e-05,
                  'huberregressor__warm_start': False,
                  'memory': None,
                  'standardscaler': StandardScaler(),
                  'standardscaler__copy': True,
                  'standardscaler__with_mean': True,
                  'standardscaler__with_std': True,
                  'steps': [('standardscaler', StandardScaler()),
                            ('huberregressor', HuberRegressor(epsilon=3))],
                  'verbose': False},
 'n_splits': 10,
 'random_seed'

,Train_start,Train_end,Train_support,Test_start,Test_end,Test_support,MSE,MSE_SE,MSE_CI_lo,MSE_CI_hi,MedAE,MedAE_SE,MedAE_CI_lo,MedAE_CI_hi,n_thresh_error_1,n_thresh_error_3,n_thresh_error_5,d25_support,MSE_d25,MedAE_d25,n_thresh_error_1_d25,n_thresh_error_3_d25,n_thresh_error_5_d25,d29_support,MSE_d29,MedAE_d29,n_thresh_error_1_d29,n_thresh_error_3_d29,n_thresh_error_5_d29,d31_support,MSE_d31,MedAE_d31,n_thresh_error_1_d31,n_thresh_error_3_d31,n_thresh_error_5_d31
0,2021-03-09 08:37:50,2021-04-15 16:17:22,5993,2021-04-15 16:47:39,2021-05-19 00:20:41,5982,61.209629,0.956163,59.542686,63.266673,5.900370,0.070687,5.765620,6.049145,0.894015,0.738883,0.570879,82,197.309823,13.110923,1.000000,1.000000,0.975610,3162,55.583755,5.376617,0.871600,0.691967,0.526249,2738,63.630667,6.195231,0.916728,0.785245,0.610299
1,2021-03-09 08:37:50,2021-05-19 00:20:41,11976,2021-05-19 00:34:07,2021-06-24 14:02:50,5982,52.717481,0.690758,51.417349,54.034873,5.907725,0.092963,5.731426,6.073620,0.945670,0.767636,0.577399,0,NaN,NaN,NaN,NaN,NaN,2533,39.725285,5.135103,0.937623,0.727991,0.512436,3449,62.259155,6.521258,0.951580,0.796753,0.625109
2,2021-03-09 08:37:50,2021-06-24 14:02:50,17959,2021-06-24 14:33:55,2021-08-08 10:58:22,5982,9.649408,0.215279,9.258893,10.093470,1.982118,0.036967,1.919352,2.067627,0.705951,0.325811,0.082247,2291,9.508275,2.082469,0.711043,0.347883,0.089481,1140,9.726499,2.154511,0.743860,0.330702,0.052632,2551,9.741706,1.877832,0.684437,0.303802,0.088985
3,2021-03-09 08:37:50,2021-08-08 10:58:22,23942,2021-08-08 11:05:07,2021-09-04 09:20:46,5977,8.098562,0.169686,7.767669,8.415198,1.880059,0.028118,1.827480,1.938414,0.734649,0.264849,0.072779,2567,8.220362,1.879696,0.735099,0.269575,0.075185,712,6.767807,1.966483,0.769663,0.245787,0.043539,2698,8.333862,1.847803,0.724981,0.265382,0.078206
4,2021-03-09 08:37:50,2021-09-04 09:20:46,29925,2021-09-04 09:31:09,2021-10-03 06:50:20,5982,4.295678,0.083652,4.138287,4.475453,1.388412,0.018098,1.353761,1.423589,0.640421,0.145436,0.018556,2678,4.658868,1.420491,0.654593,0.153099,0.025392,1166,3.780216,1.284037,0.613208,0.132075,0.006861,2138,4.121875,1.389423,0.637512,0.143124,0.016370
5,2021-03-09 08:37:50,2021-10-03 06:50:20,35908,2021-10-03 07:02:34,2021-11-02 19:53:13,5982,19.910791,0.446651,19.035117,20.783126,2.778235,0.029448,2.718651,2.834018,0.820963,0.456871,0.198930,2860,20.075540,2.699020,0.815035,0.445455,0.198951,1063,26.463967,3.088753,0.843838,0.517404,0.240828,2059,16.298742,2.711591,0.817387,0.441476,0.177271
6,2021-03-09 08:37:50,2021-11-02 19:53:13,41891,2021-11-02 20:03:37,2021-11-28 01:32:20,5980,19.495957,0.366302,18.773697,20.220031,2.101624,0.041402,2.019953,2.173656,0.718896,0.388294,0.256020,2460,21.659886,2.126359,0.730081,0.393089,0.271951,1060,19.756846,2.076688,0.687736,0.414151,0.274528,2460,17.219611,2.063263,0.721138,0.372358,0.232114
7,2021-03-09 08:37:50,2021-11-28 01:32:20,47874,2021-11-28 01:49:55,2021-12-28 14:04:13,5981,19.204482,0.238286,18.727375,19.642889,3.612378,0.048249,3.520574,3.715986,0.839659,0.579167,0.324528,2129,24.408307,4.317714,0.882104,0.657116,0.422264,1089,18.168126,3.648389,0.820018,0.586777,0.301194,2763,15.603197,3.117536,0.814694,0.516106,0.258415
8,2021-03-09 08:37:50,2021-12-28 14:04:13,53857,2021-12-28 14:17:50,2022-02-02 19:03:33,5981,13.236964,0.253155,12.764525,13.730855,2.325874,0.038402,2.253267,2.408967,0.777629,0.395586,0.160843,1414,11.848855,2.380318,0.787836,0.404526,0.135785,1237,10.432346,2.081511,0.747777,0.341956,0.112369,3330,14.868224,2.447637,0.784384,0.411712,0.189489
9,2021-03-09 08:37:50,2022-02-02 19:03:33,59840,2022-02-02 19:35:01,2022-03-13 19:01:38,5982,11.361037,0.177177,11.034423,11.691071,2.486359,0.030802,2.436139,2.550179,0.800568,0.401036,0.137078,1229,13.307780,2.818972,0.831570,0.466233,0.162734,1041,12.462838,2.808858,0.855908,0.457253,0.136407,3712,10.407501,2.297619,0.774784,0.363685,0.128772
